## Projeto final | E-commerce

### Introdução

#### E-Commerce - Identificação dos perfis dos consumidores

A análise de dados de comportamento de consumidores é uma ferramenta fundamental para empresas de e-commerce que buscam criar estratégias mais personalizadas e eficazes. A segmentação de usuários com base no histórico de compras permite que as empresas ofereçam ofertas direcionadas, aumentem a retenção de clientes e aprimorem a experiência do consumidor.

Neste projeto, serão analisados os dados de transações da Everything Plus, uma loja online que vende utensílios domésticos, com o objetivo de identificar diferentes perfis de consumidores. A análise exploratória de dados será realizada para entender os padrões de comportamento de compra. Posteriormente, os usuários serão segmentados com base nas mercadorias adquiridas, buscando explorar como diferentes categorias de produtos influenciam a segmentação. Além disso, hipóteses estatísticas serão formuladas e testadas para validar insights sobre os perfis de consumidores e suas preferências de compra.

O conjunto de dados analisado contém informações detalhadas sobre as transações realizadas, incluindo identificadores de pedidos e clientes, descrição e quantidade de itens comprados, datas das transações e os preços unitários dos produtos.

Apresentação das conclusões principais - URL: https://drive.google.com/file/d/1SRt9Z_W07v_9fY5iBMwuWOVns8DoyaAQ/view?usp=sharing

O projeto será realizado em 6 etapas:

* **Etapa 1 - Pré-processamento dos dados**: Carregar os dados e garantir o formato correto, verificar a qualidade dos dados (valores ausentes, duplicidades e inconsistências), tratar valores ausentes, ajustar tipos de dados, e identificar ou tratar outliers.

* **Etapa 2 - Análise Exploratória dos Dados (AED)**: Realizar análise descritiva de variáveis principais, explorar vendas ao longo do tempo para padrões sazonais, identificar devoluções por transações negativas, analisar variações de preço e identificar produtos mais vendidos.

* **Etapa 3 - Análise de Coorte e Métricas**: Realizar análise de coortes para medir a retenção de clientes e calcular métricas importantes como taxa de retenção, quantidade total de pedidos, valor total médio, valor unitário médio e LTV (Lifetime Value) médio. Essa etapa fornece insights sobre o comportamento dos clientes ao longo do tempo e auxilia na identificação de tendências de engajamento e fidelização.

* **Etapa 4 - Segmentação de clientes**: Utilizar K-means para segmentar clientes em clusters, analisar e interpretar os perfis identificados, e avaliar a qualidade e utilidade dos clusters para estratégias.

* **Etapa 5 - Testes de hipóteses estatísticas**: Testar hipóteses sobre comportamentos de compra, como a relação entre pedidos grandes e devoluções, impacto de devoluções na saída de clientes, sazonalidade em vendas, e correlação entre valor gasto e quantidade comprada.

* **Etapa 6 - Conclusões e recomendações**: Resumir descobertas chave, sugerir estratégias de retenção e ações com base nos perfis de clientes, como campanhas direcionadas para melhorar a fidelização.

#### Dicionário de Dados

Arquivo **ecommerce_dataset_us.csv** - contém o histórico de transações da Everything Plus, uma loja online que vende utensílios domésticos. O intervalo de datas registrado vai de 29 de novembro de 2018 até 7 de dezembro de 2019.

* `InvoiceNo` — identificador de pedido
* `StockCode` — identificador de item
* `Description` — nome de item
* `Quantity` — quantidade de itens vendidos
* `InvoiceDate` — data do pedido
* `UnitPrice` — preço por item
* `CustomerID` — identificador do cliente

### Etapa 1 - Pré-processamento dos dados

In [ ]:
# importar as bibliotecas necessárias
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
from scipy import stats as st
from scipy.stats import chi2_contingency, ttest_ind
from scipy.stats import ttest_ind, mannwhitneyu
from scipy.stats import spearmanr
from scipy.stats import kstest
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.cluster import KMeans
from wordcloud import WordCloud
from sklearn.preprocessing import StandardScaler

In [ ]:
# leitura do arquivo ecommerce_dataset_us.csv
ecommerce_dataset = pd.read_csv('ecommerce_dataset_us.csv', sep='\t')

In [ ]:
# verifica informações gerais do conjunto de dados ecommerce_dataset
ecommerce_dataset.info(memory_usage='deep')

In [ ]:
# visualiza dos dados de ecommerce_dataset
ecommerce_dataset.head(10)

In [ ]:
# exibe a quantidade de valores ausentes e a porcentagem correspondente
missing_data = ecommerce_dataset.isna().sum()
missing_percentage = (missing_data / len(ecommerce_dataset)) * 100

# cria uma tabela com a quantidade de valores ausentes e a porcentagem
missing_data_summary = pd.DataFrame({'Valores ausentes': missing_data, 'Porcentagem (%)': missing_percentage.round(2)})

# exibe o resultado
missing_data_summary

In [ ]:
# verifica a descrição dos dados
ecommerce_dataset.describe()

In [ ]:
# contagem de duplicatas
n_duplicated = ecommerce_dataset[ecommerce_dataset.duplicated()].shape[0]

print(f'O conjunto de dados possui {n_duplicated} linhas duplicadas.')

O conjunto de dados analisado contém 541.909 registros distribuídos em 7 colunas: 'InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice' e 'CustomerID', com um consumo de memória de 157.9 MB. Os nomes das colunas serão convertidos para o formato _snake_case_ para seguir as boas práticas de programação.

Foram identificados valores ausentes no conjunto de dados, com destaque para a coluna 'CustomerID', que possui cerca de 135.080 valores nulos, representando aproximadamente 25% do total de registros. Isso sugere a ocorrência de transações sem identificação de clientes. A coluna 'Description' também apresenta 1.454 valores ausentes, correspondendo a 0,27% dos dados. No futuro, esses valores serão analisados para decidir o melhor tratamento, considerando opções como preenchimento ou exclusão.

Inconsistências nos tipos de dados também foram detectadas. A coluna 'InvoiceDate', atualmente no formato _object_, será convertida para o formato _datetime_, permitindo análises temporais mais precisas. Além disso, a coluna 'CustomerID', que está no formato _float_, será ajustada para o formato _int_, uma vez que se trata de um identificador numérico. Serão também criadas novas colunas que isolem a data sem o registro de hora e o mês/ano das transações.

Valores negativos foram encontrados nas colunas 'Quantity' e 'UnitPrice', tais anomalias serão analisadas mais profundamente ao longo do projeto.

Por fim, o conjunto de dados possui 5.268 registros duplicados que, futuramente, serão examinados para decidir sobre a remoção ou tratamento adequado.

Os ajustes e análises serão realizados abaixo:

In [ ]:
# dicionário para renomear as colunas de ecommerce_dataset
rename_columns_ecommerce = {
    'InvoiceNo': 'invoice_no',
    'StockCode': 'stock_code',
    'Description': 'description',
    'Quantity': 'quantity',
    'InvoiceDate': 'invoice_date',
    'UnitPrice': 'unit_price',
    'CustomerID': 'customer_id'
}

In [ ]:
# renomeia as colunas de ecommerce_dataset
ecommerce_dataset.rename(columns=rename_columns_ecommerce, inplace=True)

In [ ]:
# altera tipo de dados da colunta 'invoice_date' para datetime
ecommerce_dataset['invoice_date'] = pd.to_datetime(ecommerce_dataset['invoice_date'])

In [ ]:
# cria uma nova coluna com o mês e ano no formato 'DD/MM/YYYY'
ecommerce_dataset['invoice_date_day'] = ecommerce_dataset['invoice_date'].dt.to_period('D')

# cria uma nova coluna com o mês e ano no formato 'MM/YYYY'
ecommerce_dataset['invoice_date_month'] = ecommerce_dataset['invoice_date'].dt.to_period('M')

# cria uma nova coluna com o valor total dos itens
ecommerce_dataset['total_price'] = ecommerce_dataset['quantity'] * ecommerce_dataset['unit_price'] 

In [ ]:
# Verifica a primeira e a última data do conjunto de dados
data_min = ecommerce_dataset['invoice_date_day'].min()
data_max = ecommerce_dataset['invoice_date_day'].max()

print(f'O intervalo de datas vai de {data_min} até {data_max}.')

In [ ]:
# visualiza dos dados de ecommerce_dataset
ecommerce_dataset.head(10)

Abaixo, serão analisados os valores ausentes de 'description':

In [ ]:
# filtra os registros onde 'description' está ausente
missing_description = ecommerce_dataset[ecommerce_dataset['description'].isna()]

# visualiza dos dados de missing_description
missing_description.head(10)

In [ ]:
# verifica a descrição dos dados
missing_description.describe()

In [ ]:
# conta quantos pedidos existem onde 'description' está ausente
items_per_invoice = missing_description['invoice_no'].value_counts()

# verifica quantos pedidos têm apenas um item e quantos têm mais de um item
single_item_orders = (items_per_invoice == 1).sum()
multiple_item_orders = (items_per_invoice > 1).sum()

print(f'Pedidos com um único item sem descrição: {single_item_orders}')
print(f'Pedidos com múltiplos itens sem descrição: {multiple_item_orders}')

In [ ]:
# conta quantos itens existem onde 'description' está ausente
unique_stock_codes_missing_description = missing_description['stock_code'].nunique()

print(f"Quantidade de 'stock_code' únicos sem descrição: {unique_stock_codes_missing_description}")

In [ ]:
# calcula a correlação entre os padrões de valores ausentes nas colunas do dataset
missing_correlation = ecommerce_dataset.isna().corr()

# exibe a correlação entre os valores ausentes da coluna 'description' e as demais colunas
missing_correlation['description']

In [ ]:
# define função para calcular a variância de dois grupos
def define_equal_var(group1, group2, threshold=0.5):
    """
    Função para calcular as variâncias de dois grupos e determinar se 'equal_var' deve ser True ou False
    com base na diferença percentual entre as variâncias.

    Parâmetros:
    - group1: array ou lista de dados do primeiro grupo
    - group2: array ou lista de dados do segundo grupo
    - threshold: limite percentual para decidir se as variâncias são diferentes o suficiente
      para usar 'equal_var=False'. O valor padrão é 0.5 (50%).

    Retorna:
    - equal_var: Boolean indicando se as variâncias podem ser consideradas iguais (True) ou diferentes (False).
    """
    # calcula as variâncias dos dois grupos
    var_group1 = np.var(group1, ddof=1)  # ddof=1 para calcular a variância amostral
    var_group2 = np.var(group2, ddof=1)

    print(f"Variância do grupo 1: {var_group1}")
    print(f"Variância do grupo 2: {var_group2}")

    # compara as variâncias usando o limiar (threshold)
    if abs(var_group1 - var_group2) / max(var_group1, var_group2) > threshold:
        print("As variâncias são bastante diferentes. equal_var=False.")
        return False
    else:
        print("As variâncias são semelhantes. equal_var=True.")
        return True

In [ ]:
# define os dois grupos de acordo com a presença ou ausência de 'description'
group1 = ecommerce_dataset[ecommerce_dataset['description'].isna()]['unit_price'].dropna()
group2 = ecommerce_dataset[~ecommerce_dataset['description'].isna()]['unit_price'].dropna()

# usa a função para determinar o valor de 'equal_var'
equal_var = define_equal_var(group1, group2, threshold=0.5)

# realiza o teste T com base no valor retornado de 'equal_var'
t_stat, p_value = ttest_ind(group1, group2, equal_var=equal_var)

# exibe os resultados
print(f"Teste T para unit_price: t-statistic = {t_stat}, p-value = {p_value}")

In [ ]:
# define os dois grupos de acordo com a presença ou ausência de 'description'
group1 = ecommerce_dataset[ecommerce_dataset['description'].isna()]['total_price'].dropna()
group2 = ecommerce_dataset[~ecommerce_dataset['description'].isna()]['total_price'].dropna()

# usa a função para determinar o valor de 'equal_var'
equal_var = define_equal_var(group1, group2, threshold=0.5)

# realiza o teste T com base no valor retornado de 'equal_var'
t_stat, p_value = ttest_ind(group1, group2, equal_var=equal_var)

# exibe os resultados
print(f"Teste T para unit_price: t-statistic = {t_stat}, p-value = {p_value}")

In [ ]:
# define os dois grupos de acordo com a presença ou ausência de 'description'
group1 = ecommerce_dataset[ecommerce_dataset['description'].isna()]['quantity'].dropna()
group2 = ecommerce_dataset[~ecommerce_dataset['description'].isna()]['quantity'].dropna()

# usa a função para determinar o valor de 'equal_var'
equal_var = define_equal_var(group1, group2, threshold=0.5)

# realiza o teste T com base no valor retornado de 'equal_var'
t_stat, p_value = ttest_ind(group1, group2, equal_var=equal_var)

# exibe os resultados
print(f"Teste T para unit_price: t-statistic = {t_stat}, p-value = {p_value}")

In [ ]:
# filtra os registros onde 'description' está ausente e agrupa pelo mês/ano de 'invoice_date'
missing_description_by_date = missing_description.groupby('invoice_date_month').size()

# converte a série resultante para um dataframe para facilitar o uso no plotly express
missing_description_by_date = missing_description_by_date.reset_index(name='count')

# converte 'invoice_date_month' para string no formato adequado para garantir que só os meses apareçam
missing_description_by_date['invoice_date_month'] = missing_description_by_date['invoice_date_month'].dt.strftime('%b-%Y')

# cria o gráfico de barras usando plotly express para mostrar a quantidade de valores ausentes de 'description' por mês/ano
fig = px.bar(
    missing_description_by_date, 
    x='invoice_date_month', 
    y='count', 
    title="Valores ausentes de 'description' por mês/ano",
    labels={'invoice_date_month': 'Mês/Ano', 'count': 'Quantidade de valores ausentes'},
    text='count'
)

# ajusta o layout para exibir todos os valores no eixo x e definir o tamanho do gráfico
fig.update_layout(
    xaxis={'tickmode': 'linear'},  # garante que todos os valores de 'invoice_date_month' apareçam
    height=500,  # ajusta a altura do gráfico para ficar menor
    width=1100,  # ajusta a largura do gráfico
    xaxis_tickangle=-45  # rotaciona os rótulos do eixo X para melhor visualização
)

# exibe o gráfico
fig.show()

As transações sem descrição apresentam características distintas. Primeiramente, verificou-se que não há valores registrados na coluna 'customer_id' para essas transações, sugerindo uma forte relação entre a ausência de descrição e a falta de identificação do cliente. A análise de correlação reforçou essa observação, com uma associação fraca, mas positiva (0.090015), entre a ausência de 'description' e 'customer_id'. Isso indica que as transações sem descrição têm uma ligeira tendência a não estarem vinculadas a clientes identificados.

Os testes T realizados mostraram que há uma diferença significativa entre as transações com e sem descrição. As transações sem descrição têm preços unitários significativamente mais baixos (t-statistic de -35.08, p-value de 2.62e-269) e envolvem menores valores totais (t-statistic de -34.96, p-value de 2.10e-267), além de uma menor quantidade de itens (t-statistic de -3.31, p-value de 0.00093). Isso sugere que essas transações estão associadas a produtos de menor valor, com uma quantidade inferior de itens vendidos.

Além disso, todas as transações com descrição ausente são pedidos únicos, o que significa que não há pedidos com múltiplos itens sem descrição. Esses resultados sugerem que essas transações seguem um padrão específico, possivelmente relacionadas a compras rápidas, onde o cliente não forneceu os dados, ou ajustes realizados pela loja, em que detalhes como a descrição do item e a identificação do cliente podem não ter sido registrados adequadamente.

Por fim, a análise temporal dos dados mostra que os meses de março, abril e julho de 2019 apresentaram um número maior de transações sem descrição. Não foi possível observar um padrão sazonal ou operacional.

Dada a análise do conjunto de dados, identificou-se que a coluna 'description' possui aproximadamente 0,27% de valores ausentes, uma quantidade pequena em relação ao total de registros. Embora essa porcentagem seja baixa o suficiente para justificar a exclusão dos registros, optou-se por um preenchimento com "unknown".

Essa abordagem mantém a integridade do conjunto de dados e garante que as transações sem descrição sejam preservadas para análises futuras, evitando a exclusão de informações que podem ser relevantes no contexto da análise geral.

In [ ]:
# preenche os valores ausentes na coluna 'description' com 'unknown'
ecommerce_dataset['description'] = ecommerce_dataset['description'].fillna('UNKNOWN')

Agora serão analisados os valores nulos da coluna 'customer_id':

In [ ]:
# filtra os pedidos com 'customer_id' ausentes e existentes
missing_customer_id = ecommerce_dataset[ecommerce_dataset['customer_id'].isna()]
existing_customer_id = ecommerce_dataset[ecommerce_dataset['customer_id'].notna()]

# visualiza dos dados de missing_customer_id
missing_customer_id.head(10)

In [ ]:
# contagem de valores de missing_customer_id
missing_customer_id.shape[0]

print(f"O conjunto dados com o campo de 'customer_id' ausente possui {missing_customer_id.shape[0]} linhas.")

In [ ]:
# agrupa os dados pelo número do pedido ('invoice_no') e verifica se todos os 'customer_id' estão ausentes no pedido
customer_id_per_invoice = ecommerce_dataset.groupby('invoice_no')['customer_id'].apply(lambda x: x.isna().all())

# verifica se há pedidos com alguns itens com 'customer_id' ausente e outros não
mixed_customer_id = ecommerce_dataset.groupby('invoice_no')['customer_id'].apply(lambda x: x.isna().any() and x.notna().any())

# conta quantos pedidos estão completamente sem 'customer_id' e quantos têm uma mistura de itens com e sem 'customer_id'
total_missing_per_order = customer_id_per_invoice.sum()
total_mixed_per_order = mixed_customer_id.sum()

print(f"Total de pedidos onde todos os itens estão sem 'customer_id': {total_missing_per_order}")
print(f"Total de pedidos com mistura de itens com e sem 'customer_id': {total_mixed_per_order}")

In [ ]:
# agrupa os dados por 'invoice_no' e soma a quantidade total de itens e o valor total do pedido no conjunto filtrado
grouped_invoices = missing_customer_id.groupby('invoice_no').agg({
    'quantity': 'sum',  # soma a quantidade total de itens
    'total_price': 'sum'  # soma o valor total do pedido
}).reset_index()

# visualiza os dados agrupados
grouped_invoices.describe()

In [ ]:
# agrupa os dados por 'invoice_no' e soma a quantidade total de itens e o valor total do pedido no conjunto original
ecommerce_dataset.groupby('invoice_no').agg({
    'quantity': 'sum',  # soma a quantidade total de itens
    'total_price': 'sum'  # soma o valor total do pedido
}).reset_index().describe()

In [ ]:
# agrupa os dados por 'invoice_no' e soma a quantidade total de itens em cada pedido para clientes com e sem 'customer_id'
missing_customer_items = missing_customer_id.groupby('invoice_no')['quantity'].sum().reset_index(name='total_items_missing')
existing_customer_items = existing_customer_id.groupby('invoice_no')['quantity'].sum().reset_index(name='total_items_existing')

# define os dois grupos de acordo com a presença ou ausência de 'customer_id'
group1 = missing_customer_items['total_items_missing']
group2 = existing_customer_items['total_items_existing']

# usa a função para determinar o valor de 'equal_var'
equal_var = define_equal_var(group1, group2, threshold=0.5)

# realiza o teste T com base no valor retornado de 'equal_var'
t_stat, p_value = ttest_ind(group1, group2, equal_var=equal_var)

# exibe os resultados
print(f"Teste T para quantidade de itens: t-statistic = {t_stat}, p-value = {p_value}")

In [ ]:
# agrupa os dados por 'invoice_no' e calcula a média do valor unitário dos itens em cada pedido para pedidos com e sem 'customer_id'
missing_customer_unit_price = missing_customer_id.groupby('invoice_no')['unit_price'].mean().reset_index(name='avg_unit_price_missing')
existing_customer_unit_price = existing_customer_id.groupby('invoice_no')['unit_price'].mean().reset_index(name='avg_unit_price_existing')

# define os dois grupos com base na média de preços unitários por pedido para pedidos com e sem 'customer_id'
group1 = missing_customer_unit_price['avg_unit_price_missing']
group2 = existing_customer_unit_price['avg_unit_price_existing']

# usa a função para determinar o valor de 'equal_var'
equal_var = define_equal_var(group1, group2, threshold=0.5)

# realiza o teste T com base no valor retornado de 'equal_var'
t_stat_price, p_value_price = ttest_ind(group1, group2, equal_var=equal_var)

# exibe os resultados
print(f"Teste T para preço unitário: t-statistic = {t_stat_price}, p-value = {p_value_price}")


In [ ]:
# agrupa os dados por 'invoice_no' e soma o valor total do pedido para pedidos com e sem 'customer_id'
missing_customer_total_price = missing_customer_id.groupby('invoice_no')['total_price'].sum().reset_index(name='total_price_missing')
existing_customer_total_price = existing_customer_id.groupby('invoice_no')['total_price'].sum().reset_index(name='total_price_existing')

# define os dois grupos com base no valor total por pedido
group1 = missing_customer_total_price['total_price_missing']
group2 = existing_customer_total_price['total_price_existing']

# usa a função para determinar o valor de 'equal_var'
equal_var = define_equal_var(group1, group2, threshold=0.5)

# realiza o teste T com base no valor retornado de 'equal_var'
t_stat_total, p_value_total = ttest_ind(group1, group2, equal_var=equal_var)

# exibe os resultados
print(f"Teste T para valor total do pedido: t-statistic = {t_stat_total}, p-value = {p_value_total}")

In [ ]:
# filtra os 25 itens mais vendidos sem 'customer_id'
missing_customer_id[missing_customer_id['description'].notna()]['description'].value_counts().reset_index().head(25)

In [ ]:
# filtra os registros onde 'customer_id' está ausente
missing_customer_orders = ecommerce_dataset[ecommerce_dataset['customer_id'].isna()]

# agrupa os pedidos por 'invoice_no' e obtém a primeira ocorrência da data para garantir que o agrupamento seja por pedidos
missing_customer_orders_by_day = missing_customer_orders.groupby('invoice_no').first().reset_index()

# agrupa os dados pelo dia da 'invoice_date' e conta a quantidade de pedidos por dia
missing_orders_by_day = missing_customer_orders_by_day.groupby('invoice_date_day').size().reset_index(name='count')

# converte 'invoice_date_day' para string no formato adequado para garantir que só os dias apareçam
missing_orders_by_day['invoice_date_day'] = missing_orders_by_day['invoice_date_day'].dt.strftime('%d-%b-%Y')

# calcula a mediana da quantidade de pedidos por dia
median_count = missing_orders_by_day['count'].median()

In [ ]:
# cria o gráfico de barras usando plotly express para mostrar a quantidade de pedidos sem 'customer_id' por dia
fig = px.bar(
    missing_orders_by_day, 
    x='invoice_date_day', 
    y='count', 
    title="Distribuição de pedidos sem 'customer_id' por dia",
)

# adiciona a linha de mediana usando plotly.graph_objects
fig.add_trace(
    go.Scatter(
        x=missing_orders_by_day['invoice_date_day'], 
        y=[median_count] * len(missing_orders_by_day),  # valor constante igual à mediana
        mode='lines',
        name='Mediana',
        line=dict(color='red', dash='dash')  # cor e estilo da linha
    )
)

# ajusta o layout para exibir todos os valores no eixo x e definir o tamanho do gráfico
fig.update_layout(
    height=450,  # ajusta a altura do gráfico para ficar menor
    width=1200,  # ajusta a largura do gráfico
    xaxis_tickangle=-45  # rotaciona os rótulos do eixo X para melhor visualização
)

# exibe o gráfico
fig.show()



In [ ]:
# criar uma coluna temporária que indica se o valor de 'customer_id' é nulo ou não
ecommerce_dataset['customer_id_status'] = ecommerce_dataset['customer_id'].isna().map({True: 'Nulo', False: 'Não Nulo'})

# lista de colunas para gerar gráficos (excluindo customer_id)
columns_to_plot = ['invoice_date', 'stock_code', 'description', 'quantity', 'unit_price', 'total_price']

# loop para criar um gráfico para cada coluna na lista
for col in columns_to_plot:
    # Criar o gráfico de histograma para cada coluna
    fig = px.histogram(
        ecommerce_dataset, 
        x=col, 
        color='customer_id_status', 
        title=f'Distribuição de "{col}" distinguindo valores nulos e não nulos em "customer_id"',
        barmode='overlay'
    )

    # exibir o gráfico
    fig.show()

A análise dos dados ausentes da coluna 'customer_id' revelou algumas características interessantes. Ao todo, foram identificadas 135.080 transações sem a identificação de clientes, representando aproximadamente 25% do total dos dados, conforme observado anteriormente. A investigação aprofundada mostrou que 3.710 pedidos foram realizados sem 'customer_id', ou seja, todos os itens desses pedidos não tinham o campo de identificação do cliente preenchido. Não foram encontradas transações que misturassem itens com e sem 'customer_id' em um mesmo pedido.

Ao agrupar os dados por 'invoice_no' e calcular a quantidade de itens por pedido, o teste T aplicado para comparar as médias entre os grupos que contém ou não a informação de 'customer_id' indicou uma diferença significativa. O valor de t-statistic foi de -11.686 e o p-value de 2.49e-31, evidenciando que, em média, os pedidos sem 'customer_id' contêm mais itens do que os pedidos identificados. Em relação aos preços unitários, o teste T apontou uma variação significativa entre os grupos. O valor de t-statistic foi de -4.74 e o p-value de 2.14e-06, sugerindo que os itens vendidos em transações sem 'customer_id' possuem preços unitários menores. No entanto, ao analisar o valor total dos pedidos, o teste T não encontrou uma diferença estatisticamente significativa entre os grupos, com t-statistic de 0.452 e p-value de 0.65. Isso indica que o valor total dos pedidos com e sem 'customer_id' é semelhante.

Ao analisar os itens mais vendidos sem 'customer_id', foi identificado que o item mais comum é 'DOTCOM POSTAGE', que aparece com 693 ocorrências. Este item refere-se ao custo de envio das transações, e sua alta frequência sugere que muitos pedidos sem identificação de cliente incluem custos de envio destacados. Entre os itens populares mais vendidos nos registros sem 'customer_id', chamou a atenção a presença de diversas bolsas, como a 'JUMBO BAG RETROSPOT' e outras similares, indicando que produtos deste tipo são amplamente comprados em transações onde a identificação do cliente está ausente.

A análise temporal revelou que os valores ausentes de 'customer_id' variam significativamente ao longo do tempo, com alguns dias registrando picos muito acima da média, enquanto a maioria dos dias apresenta valores próximos ou abaixo dessa média. Isso sugere que, embora haja uma demanda regular, ocorrem eventos esporádicos que impulsionam um número maior de pedidos sem identificação de cliente.

A comparação entre os registros com e sem 'customer_id' reforçam os padrões observados anteriormente.

Dada a análise realizada, a melhor estratégia para tratar os valores ausentes em customer_id é imputar o valor numeral "0". Essa abordagem mantém a integridade dos dados, preserva as transações relevantes para análise e permite identificar pedidos anônimos sem comprometer o conjunto de informações, garantindo consistência nas análises futuras.

In [ ]:
# verifica se há valores '0' pré-existentes na coluna 'customer_id'
zero_count = ecommerce_dataset[ecommerce_dataset['customer_id'] == 0].shape[0]

# exibe o resultado com uma mensagem
print(f"Número de registros com 'customer_id' igual a 0: {zero_count}")

In [ ]:
# substitui valores ausentes na coluna 'customer_id' com 0
ecommerce_dataset['customer_id'] = ecommerce_dataset['customer_id'].fillna(0)

# converte os valores da coluna 'customer_id' para Int64, convertendo erros em NaN
ecommerce_dataset['customer_id'] = ecommerce_dataset['customer_id'].astype('Int64')

In [ ]:
# exibe novamente a quantidade de valores ausentes e a porcentagem correspondente
missing_data = ecommerce_dataset.isna().sum()
missing_percentage = (missing_data / len(ecommerce_dataset)) * 100

# cria uma tabela com a quantidade de valores ausentes e a porcentagem
missing_data_summary = pd.DataFrame({'Valores ausentes': missing_data, 'Porcentagem (%)': missing_percentage.round(2)})

# exibe o resultado
missing_data_summary

Agora serão analisados os valores negativos presentes nas colunas 'Quantity' e 'UnitPrice', a fim de entender sua natureza e tratá-los:

In [ ]:
# filtrar transações com quantidade negativa (possíveis devoluções)
ecommerce_dataset[ecommerce_dataset['quantity'] < 0]

In [ ]:
# filtra todos os registros de um dos usuários com transação com quantidade negativa
ecommerce_dataset[ecommerce_dataset['customer_id'] == 17548]

In [ ]:
# verifica se há valores de quantidade com invoice_no "C" maiores que 0
ecommerce_dataset[(ecommerce_dataset['invoice_no'].str.startswith('C')) & (ecommerce_dataset['quantity'] > 0)]

Foi observado que as faturas cujo número começa com a letra "C" possivelmente indicam transações relacionadas a devoluções, créditos ou descontos, representando operações que impactam negativamente o estoque e o valor total das vendas. Para facilitar a análise desses registros e possibilitar um entendimento mais claro das transações, será criada uma nova coluna no conjunto de dados para categorizar essas transações de acordo com sua natureza. 

A proposta é identificar tais transações como "Credit":

In [ ]:
# cria a coluna 'transaction_type' com valores nulos inicialmente
ecommerce_dataset['transaction_type'] = None

In [ ]:
# atribui "Credit" para todas as transações em que 'invoice_no' começa com "C"
ecommerce_dataset.loc[ecommerce_dataset['invoice_no'].str.startswith('C'), 'transaction_type'] = 'Credit'

A análise se estendeu para outras possibilidades de transações. Abaixo, foram identificadas transações com valores negativos na coluna 'quantity' que não possuem a descrição 'UNKNOWN' e cujas 'invoice_no' não começam com a letra 'C'. Após análise, foi definido que essas transações serão marcadas com o tipo "Damaged/Returned Stock" na nova coluna 'transaction_type':

In [ ]:
# filtra e retorna as descrições de transações que possuem quantidade negativa, cuja 'invoice_no' não começa com 'C' e que não possuem a descrição 'UNKNOWN'
ecommerce_dataset[
    (~ecommerce_dataset['invoice_no'].str.startswith('C')) & 
    (ecommerce_dataset['quantity'] < 0) & 
    (ecommerce_dataset['description'] != 'UNKNOWN')]['description'].unique()

In [ ]:
# atribui "Damaged/Returned Stock" para todas as transações que atendam as condições especificadas
ecommerce_dataset.loc[
    (~ecommerce_dataset['invoice_no'].str.startswith('C')) & 
    (ecommerce_dataset['quantity'] < 0) & 
    (ecommerce_dataset['description'] != 'UNKNOWN'), 
    'transaction_type'
] = 'Damaged/Returned Stock'

In [ ]:
# verifica se há valores negativos da coluna 'quantity' sem categoria em 'transaction_type'
ecommerce_dataset[(ecommerce_dataset['quantity'] < 0) & (ecommerce_dataset['transaction_type'] == None)]

A análise foi aprofundada para identificar outras transações com valores negativos, desta vez focando na coluna 'unit_price'. As transações que apresentaram valores negativos na coluna 'unit_price' foram consideradas ajustes de dívida, pois esses registros refletem correções financeiras, como ajustes de débitos. As transações foram categorizadas como 'Debt Adjustment' na nova coluna 'transaction_type':

In [ ]:
# filtrar transações com quantidade negativa
ecommerce_dataset[ecommerce_dataset['unit_price'] < 0]

In [ ]:
# atribui "Debt Adjustment" para todas as transações que atendam as condições especificadas
ecommerce_dataset.loc[(ecommerce_dataset['unit_price'] < 0), 'transaction_type'] = 'Debt Adjustment'

Além disso, os itens com descrição 'UNKNOWN' terão seu tipo de transação registrados como 'Uncategorized', uma vez que não foi possível observar padrões nos dados:

In [ ]:
# atribui "Uncategorized" para todas as transações que atendam as condições especificadas
ecommerce_dataset.loc[
    (ecommerce_dataset['transaction_type'].isna()) & 
    (ecommerce_dataset['description'] == 'UNKNOWN'),
    'transaction_type'
] = 'Uncategorized'

Conforme observado, os valores negativos desempenham um papel importante nas transações analisadas, por isso, não serão alterados diretamente. A categorização desses valores permitirá uma identificação rápida e clara do tipo de transação.

In [ ]:
# verifica se há valores com quantidades negativas sem categorização
ecommerce_dataset[(ecommerce_dataset['transaction_type'].isna()) & (ecommerce_dataset['quantity'] < 0)]

In [ ]:
# verifica se há valores com quantidades negativas sem categorização
ecommerce_dataset[(ecommerce_dataset['transaction_type'].isna()) & (ecommerce_dataset['unit_price'] < 0)]

A seguir, as transações com valores positivos também serão categorizadas. Para tais dados, a categoria registrada será 'Regular transaction':

In [ ]:
# atribui "Uncategorized" para todas as transações que atendam as condições especificadas
ecommerce_dataset.loc[
    (ecommerce_dataset['transaction_type'].isna()) & 
    (ecommerce_dataset['total_price'] >= 0),
    'transaction_type'
] = 'Regular transaction'

In [ ]:
# atribui "Regular transaction" para todas as transações que atendam as condições especificadas
ecommerce_dataset.loc[ecommerce_dataset['total_price'] > 0, 'transaction_type'] = 'Regular transaction'

In [ ]:
# verifica se todos os dados foram devidamente categorizados
ecommerce_dataset[ecommerce_dataset['transaction_type'].isna()]

Para completar o pré-processamento dos dados, será realizada a identificação e o tratamento dos dados duplicados, assegurando que cada transação seja única e válida para as próximas etapas da análise:

In [ ]:
# remoção de duplicatas
ecommerce_dataset.drop_duplicates(inplace=True)

In [ ]:
# verificação de duplicatas
n_duplicated = ecommerce_dataset[ecommerce_dataset.duplicated()].shape[0]

print(f'Após o tratamento, o conjunto de dados possui {n_duplicated} linhas duplicadas.')

### Etapa 2 - Análise exploratória dos dados (AED)

Aqui serão exploradas relações entre variáveis, a distribuição de valores e possíveis correlações entre as diferentes colunas, com o objetivo de identificar insights iniciais sobre o comportamento de compra dos consumidores. A análise exploratória dos dados ajudará a guiar as próximas etapas da análise, destacando tendências, anomalias e possíveis perfis de clientes que poderão ser segmentados com base nos padrões observados.

Nesta primeira sessão, será realizada a análise da contagem de pedidos mensais, observando o número de pedidos únicos registrados em cada mês a partir dos registros com tipo de transação 'Regular'. A análise permitirá identificar o volume de pedidos ao longo do tempo, com foco nas variações mensais e na detecção de possíveis padrões sazonais.

In [ ]:
# filtra os registros onde 'transaction_type' é "Regular transaction"
regular_transactions = ecommerce_dataset[ecommerce_dataset['transaction_type'] == 'Regular transaction']

In [ ]:
# Conta o número de pedidos únicos (invoice_no) em cada mês
monthly_invoices = regular_transactions.groupby('invoice_date_month')['invoice_no'].nunique().reset_index()

# Converte 'invoice_date_month' de volta para string para ser usado no gráfico
monthly_invoices['invoice_date_month'] = monthly_invoices['invoice_date_month'].astype(str)

# Cria o histograma
fig = px.line(
    monthly_invoices,
    x='invoice_date_month',
    y='invoice_no',
    title='Total de pedidos por mês',
    labels={'invoice_date_month': 'Mês/Ano', 'invoice_no': 'Contagem de pedidos'},
)

# Ajusta o layout para melhor visualização
fig.update_layout(
    xaxis_title="Mês/Ano",
    yaxis_title="Número de pedidos",
    xaxis_tickangle=-45,
    xaxis = dict(
        tickvals = monthly_invoices['invoice_date_month'],  # posição dos ticks
        ticktext = monthly_invoices['invoice_date_month'],  # texto dos ticks
        tickangle = -45  # rotaciona os ticks
    ),
    height=450,
    width=1000
)

# Exibe o gráfico
fig.show()


O gráfico "Total de pedidos por mês" mostra uma tendência de crescimento geral ao longo do período, com variações significativas em alguns meses. De janeiro a junho de 2019, observa-se uma flutuação no número de pedidos, com picos e quedas que sugerem variações mensais na demanda e possivelmente influências sazonais ou promocionais. Esse comportamento flutuante ao longo do primeiro semestre contrasta com uma leve estabilidade a partir de julho, que então culmina em um pico acentuado em novembro de 2019, provavelmente associado a promoções de fim de ano ou eventos de alta demanda. A queda em dezembro de 2019 não reflete necessariamente uma redução na atividade comercial, mas se deve ao fato de que os registros foram capturados apenas até o início do mês, limitando os dados disponíveis. 

A próxima sessão examinará o comportamento das quantidades de itens por transação e as variações temporais das vendas. Inicialmente, será analisada a distribuição da quantidade de itens por pedido, com foco na quantidade média de itens e na identificação de pedidos com volumes atípicos. Em seguida, a análise temporal irá explorar as vendas ao longo do tempo, identificando picos e variações mensais. Por fim, as transações com quantidades negativas serão avaliadas, destacando devoluções e cancelamentos.

In [ ]:
# calcula os limites inferior e superior para outliers em 'quantity'
Q1_quantity, Q3_quantity = regular_transactions['quantity'].quantile([0.25, 0.75])
IQR_quantity = Q3_quantity - Q1_quantity
lower_bound_quantity = Q1_quantity - 1.5 * IQR_quantity
upper_bound_quantity = Q3_quantity + 1.5 * IQR_quantity

# filtra os registros que não são outliers em 'quantity'
regular_transactions_quantity_without_outliers = regular_transactions[(regular_transactions['quantity'] >= lower_bound_quantity) & (regular_transactions['quantity'] <= upper_bound_quantity)]

In [ ]:
# cria uma box plot horizontal para os valores da coluna 'quantity' apenas para "Regular transaction"
fig = px.box(
    regular_transactions_quantity_without_outliers, 
    x='quantity',  # Define 'quantity' no eixo X para um gráfico horizontal
    title='Box plot dos valores de quantidade para Regular transaction sem outliers',
    labels={'quantity': 'Quantidade'}
)

# ajusta o layout
fig.update_layout(
    height=350,  # ajusta a altura do gráfico para ficar menor
    width=1000,  # ajusta a largura do gráfico
)

# exibe o gráfico
fig.show()

In [ ]:
# cria uma box plot horizontal para os valores da coluna 'quantity' apenas para "Regular transaction"
fig = px.box(
    regular_transactions, 
    x='quantity',  # Define 'quantity' no eixo X para um gráfico horizontal
    title='Box plot dos valores de quantidade para Regular transaction com outliers',
    labels={'quantity': 'Quantidade'}
)

# ajusta o layout
fig.update_layout(
    height=350,  # ajusta a altura do gráfico para ficar menor
    width=1000,  # ajusta a largura do gráfico
)

# exibe o gráfico
fig.show()

A análise da distribuição de quantidades para transações regulares, sem a presença de outliers, revela que a maior parte dos pedidos contém entre 1 e 10 itens. A mediana, com o valor de 3 itens, destaca que metade das transações envolve essa quantidade ou menos. O gráfico sem outliers também mostra que, embora sejam menos comuns, há algumas transações com até 26 itens. A análise foi realizada considerando a ausência de outliers, o que permite uma visualização mais precisa da tendência central e da dispersão dos dados, reforçando que a maioria das transações se concentra em volumes menores. Já o gráfico com outliers revela uma dispersão muito maior, com algumas transações chegando a quase 81.000 itens, mostrando a extrema variação nos volumes de pedidos.

In [ ]:
# agrupa os dados pelo mês e soma a quantidade de itens vendidos em cada mês
monthly_sales_quantity = ecommerce_dataset.groupby('invoice_date_month')['quantity'].sum().reset_index()

# converte a coluna 'invoice_date_month' para string
monthly_sales_quantity['invoice_date_month'] = monthly_sales_quantity['invoice_date_month'].astype(str)

# cria um gráfico de linha para visualizar as vendas mensais
fig = px.line(monthly_sales_quantity, 
              x='invoice_date_month', 
              y='quantity', 
              title='Quantidade de itens vendidos mensalmente ao longo do tempo',
              labels={'invoice_date_month': 'Mês/Ano', 'quantity': 'Quantidade de itens vendidos'})

# ajusta o layout do gráfico
fig.update_layout(
    xaxis_title='Mês/Ano',
    yaxis_title='Quantidade de itens vendidos',
    xaxis = dict(
        tickvals = monthly_sales_quantity['invoice_date_month'],  # posição dos ticks
        ticktext = monthly_sales_quantity['invoice_date_month'],  # texto dos ticks
        tickangle = -45  # rotaciona os ticks
    ),    
    height=450,  # ajusta a altura do gráfico para ficar menor
    width=1000  # ajusta a largura do gráfico
)

# exibe o gráfico
fig.show()

In [ ]:
# agrupa os dados pelo mês e soma a quantidade de itens vendidos em cada mês
monthly_sales_total_price = ecommerce_dataset.groupby('invoice_date_month')['total_price'].sum().reset_index()

# converte a coluna 'year_month' para string
monthly_sales_total_price['invoice_date_month'] = monthly_sales_total_price['invoice_date_month'].astype(str)

# cria um gráfico de linha para visualizar as vendas mensais
fig = px.line(monthly_sales_total_price, 
              x='invoice_date_month', 
              y='total_price', 
              title='Arrecadação mensal ao longo do tempo',
              labels={'invoice_date_month': 'Mês/Ano', 'total_price': 'Valor arrecadado'})

# ajusta o layout do gráfico
fig.update_layout(
    xaxis_title='Mês/Ano',
    yaxis_title='Valor arrecadado',
    xaxis = dict(
        tickvals = monthly_sales_total_price['invoice_date_month'],  # posição dos ticks
        ticktext = monthly_sales_total_price['invoice_date_month'],  # texto dos ticks
        tickangle = -45  # rotaciona os ticks
    ),
    height=450,  # ajusta a altura do gráfico para ficar menor
    width=1000,  # ajusta a largura do gráfico
)

# exibe o gráfico
fig.show()

A análise temporal das vendas ao longo do tempo, considerando o intervalo de 29 de novembro de 2018 a 7 de dezembro de 2019, revela uma tendência de crescimento na quantidade de itens vendidos e na arrecadação mensal ao longo de 2019. Observa-se que, após uma queda inicial, as vendas começam a crescer gradualmente a partir de março de 2019, com picos notáveis em novembro de 2019, possivelmente devido a promoções sazonais como Black Friday. Em seguida, há uma queda acentuada em dezembro, que pode estar relacionada ao período de coleta de dados mais curto no mês. Essa variação sazonal é consistente tanto para a quantidade de itens vendidos quanto para o valor arrecadado, destacando a influência de datas específicas nas vendas.

In [ ]:
# filtra as transações com quantidade negativa (possíveis devoluções ou cancelamentos)
negative_transactions = ecommerce_dataset[ecommerce_dataset['quantity'] < 0]

# agrupa os dados por 'invoice_date_month' e soma a quantidade de itens devolvidos/cancelados em cada mês
monthly_returns = negative_transactions.groupby(['invoice_date_month', 'transaction_type'])['quantity'].sum().reset_index()

# converte a coluna 'invoice_date_month' para string
monthly_returns['invoice_date_month'] = monthly_returns['invoice_date_month'].astype(str)

In [ ]:
# cria o gráfico de barras empilhadas para visualizar devoluções/cancelamentos por mês e categoria de transação
fig = px.bar(
    monthly_returns, 
    x='invoice_date_month', 
    y='quantity', 
    color='transaction_type',  # divide as barras pelas categorias de transação
    title='Devoluções e cancelamentos por mês e tipo de transação',
    labels={'invoice_date_month': 'Mês/Ano', 'quantity': 'Quantidade de itens'},
    barmode='stack',  # ajusta para barras empilhadas
)

# ajusta o layout do gráfico
fig.update_layout(
    xaxis_title='Mês/Ano',
    yaxis_title='Quantidade de itens devolvidos/cancelados',
    legend_title='Tipo de transação',  # título da legenda
    xaxis = dict(
        tickvals = monthly_returns['invoice_date_month'],  # posição dos ticks
        ticktext = monthly_returns['invoice_date_month'],  # texto dos ticks
        tickangle = -45  # rotaciona os ticks
    ),    
    height=500,  # ajusta a altura do gráfico para ficar menor
    width=1100  # ajusta a largura do gráfico
)

# exibe o gráfico
fig.show()

O gráfico mostra a quantidade de devoluções e cancelamentos mensais, categorizados pelos tipos de transação: Credit, Damaged/Returned Stock e Uncategorized. Observa-se um aumento significativo nas devoluções em janeiro e dezembro de 2019, sendo o tipo Credit o principal responsável por esses picos. Esses meses podem estar relacionados a um momento de pós-eventos sazonais. A categoria Damaged/Returned Stock também apresenta contribuições relevantes, especialmente em meses como junho de 2019 e outubro de 2019, onde se observa um aumento de produtos devolvidos ou danificados.

Nesta seção, será investigada a distribuição dos preços unitários, identificando produtos com preços muito altos ou muito baixos, que podem representar outliers. Além disso, será calculado o total gasto por pedido, com o objetivo de avaliar a relação entre o número de itens adquiridos e o valor total da compra.

In [ ]:
# calcula os limites inferior e superior para outliers em 'unit_price'
Q1_unit_price, Q3_unit_price = regular_transactions['unit_price'].quantile([0.25, 0.75])
IQR_unit_price = Q3_unit_price - Q1_unit_price
lower_bound_unit_price = Q1_unit_price - 1.5 * IQR_unit_price
upper_bound_unit_price = Q3_unit_price + 1.5 * IQR_unit_price

# filtra os registros que não são outliers em 'unit_price'
regular_transactions_unit_price_without_outliers = regular_transactions[(regular_transactions['unit_price'] >= lower_bound_unit_price) & (regular_transactions['unit_price'] <= upper_bound_unit_price)]

In [ ]:
# cria uma box plot horizontal para os valores da coluna 'unit_price' apenas para "Regular transaction"
fig = px.box(
    regular_transactions_unit_price_without_outliers, 
    x='unit_price',  # Define 'unit_price' no eixo X para um gráfico horizontal
    title='Box plot dos valores unitários para Regular transaction sem outliers',
    labels={'quantity': 'Quantidade'}
)

# ajusta o layout
fig.update_layout(
    height=350,  # ajusta a altura do gráfico para ficar menor
    width=1000,  # ajusta a largura do gráfico
)

# exibe o gráfico
fig.show()

In [ ]:
# cria uma box plot horizontal para os valores da coluna 'unit_price' apenas para "Regular transaction"
fig = px.box(
    ecommerce_dataset, 
    x='unit_price',  # Define 'unit_price' no eixo X para um gráfico horizontal
    title='Box plot dos valores unitários para Regular transaction com outliers',
    labels={'quantity': 'Quantidade'}
)

# ajusta o layout
fig.update_layout(
    height=350,  # ajusta a altura do gráfico para ficar menor
    width=1000,  # ajusta a largura do gráfico
)

# exibe o gráfico
fig.show()

No gráfico sem outliers, observa-se que a maioria dos preços unitários dos produtos está concentrada entre 1 e 4 unidades monetárias, com a mediana em aproximadamente 2. Isso indica que a maior parte dos produtos vendidos possui um valor acessível dentro dessa faixa de preço. Já no gráfico que inclui outliers, é possível visualizar uma dispersão muito maior, com valores negativos já identificados anteriormente como correções financeiras. Além disso, são notáveis valores unitários extremos, chegando até aproximadamente 40 mil unidades monetárias, o que claramente se destaca do padrão observado. Esses outliers podem indicar erros, ajustes financeiros ou itens específicos de alto valor, que precisam de uma investigação mais detalhada. Em geral, a distribuição de preços unitários se concentra em valores baixos, mas a presença de outliers sugere uma diversidade maior de comportamentos de preço nas transações.

In [ ]:
# calcula os limites inferior e superior para outliers em 'total_price'
Q1_total_price, Q3_total_price = regular_transactions['total_price'].quantile([0.25, 0.75])
IQR_total_price = Q3_total_price - Q1_total_price
lower_bound_total_price = Q1_total_price - 1.5 * IQR_total_price
upper_bound_total_price = Q3_total_price + 1.5 * IQR_total_price

# filtra os registros que não são outliers em 'total_price'
regular_transactions_without_total_price_outliers = regular_transactions[(regular_transactions['total_price'] >= lower_bound_total_price) & (regular_transactions['total_price'] <= upper_bound_total_price)]

In [ ]:
# cria uma box plot horizontal para os valores de total_price sem outliers
fig = px.box(
    regular_transactions_without_total_price_outliers,
    x='total_price',
    title='Box plot dos valores de total_price para Regular transaction sem outliers',
    labels={'total_price': 'Preço total'}
)

# ajusta o layout
fig.update_layout(
    height=400,  # ajusta a altura do gráfico para ficar menor
    width=1000   # ajusta a largura do gráfico
)

# Exibe o gráfico
fig.show()

In [ ]:
# cria uma box plot horizontal para os valores de total_price com outliers
fig = px.box(
    regular_transactions,
    x='total_price',
    title='Box plot dos valores de total_price para Regular transaction com outliers',
    labels={'total_price': 'Preço total'}
)

# ajusta o layout
fig.update_layout(
    height=400,  # ajusta a altura do gráfico para ficar menor
    width=1000   # ajusta a largura do gráfico
)

# exibe o gráfico
fig.show()

O primeiro gráfico, sem outliers, mostra que a maioria dos pedidos tem um valor total entre aproximadamente 3 e 16 unidades monetárias, com a mediana em torno de 8.32. Isso indica que metade das transações registra um valor abaixo deste valor. O intervalo interquartil (IQR) também confirma uma distribuição concentrada até cerca de 34 unidades monetárias. No entanto, ao incluir os outliers no segundo gráfico, nota-se uma grande dispersão nos valores, com alguns pedidos chegando a valores extremamente altos, como 160 mil unidades monetárias. Esses outliers representam uma pequena fração das transações, mas indicam a presença de pedidos com valores excepcionalmente altos.

Nesta sessão, serão avaliados os produtos mais comprados, tanto em termos de quantidade de unidades vendidas quanto em receita total gerada. 

In [ ]:
# agrupa os dados por produto e soma a quantidade de itens vendidos e o total de receita gerada
product_sales = ecommerce_dataset.groupby('description').agg(
    total_quantity=('quantity', 'sum'),
    total_revenue=('total_price', 'sum')
).reset_index()

# ordena os produtos pela quantidade total vendida
top_products_by_quantity = product_sales.sort_values(by='total_quantity', ascending=False).head(10)

# ordena os produtos pela receita total gerada
top_products_by_revenue = product_sales.sort_values(by='total_revenue', ascending=False).head(10)

# cria um gráfico de barras horizontal para visualizar os 10 produtos mais vendidos em termos de quantidade
fig_quantity = px.bar(top_products_by_quantity, 
                     y='description', 
                     x='total_quantity', 
                     title='Top 10 produtos mais vendidos por quantidade',
                     labels={'description': 'Produto', 'total_quantity': 'Quantidade Total Vendida'})

# Inverte a ordem do eixo y para que o maior valor fique no topo
fig_quantity.update_yaxes(categoryorder='total ascending')

# ajusta o layout do gráfico
fig_quantity.update_layout(
    height=500,  # ajusta a altura do gráfico para visualização
    width=1100   # ajusta a largura do gráfico
)

# exibe o gráfico
fig_quantity.show()

# cria um gráfico de barras horizontal para visualizar os 10 produtos que mais geraram receita
fig_revenue = px.bar(top_products_by_revenue, 
                    y='description', 
                    x='total_revenue', 
                    title='Top 10 produtos que mais geraram receita',
                    labels={'description': 'Produto', 'total_revenue': 'Receita Total Gerada'})

# Inverte a ordem do eixo y para que o maior valor fique no topo
fig_revenue.update_yaxes(categoryorder='total ascending')

# ajusta o layout do gráfico
fig_revenue.update_layout(
    height=500,  # ajusta a altura do gráfico para visualização
    width=1100   # ajusta a largura do gráfico
)

# exibe o gráfico
fig_revenue.show()

Analisando os dois gráficos, é possível observar alguns padrões entre os produtos mais vendidos e os que geraram mais receita. No gráfico dos "Top 10 Produtos Mais Vendidos por Quantidade", muitos dos itens são produtos pequenos e de baixo valor unitário, como WORLD WAR 2 GLIDERS ASSTD DESIGNS e JUMBO BAG RED RETROSPOT, sugerindo que esses produtos são adquiridos em grandes volumes, mas com baixo impacto individual na receita.

Já no gráfico dos "Top 10 Produtos que Mais Geraram Receita", observa-se que itens como DOTCOM POSTAGE e REGENCY CAKESTAND 3 TIER não aparecem entre os mais vendidos em termos de quantidade, mas geram uma receita significativa, indicando que produtos com valores unitários mais altos ou custos adicionais (como o frete) têm um impacto relevante no faturamento.

Há também uma predominância de itens decorativos e utilitários para casa em ambas as listas, como JUMBO BAG RED RETROSPOT e WHITE HANGING HEART T-LIGHT HOLDER, sugerindo uma forte demanda por esses tipos de produtos.

### Etapa 3 - Análise de coorte e métricas

A Análise de Coorte e Métricas tem como objetivo examinar o comportamento de compra dos clientes ao longo do tempo, segmentando-os por coortes de aquisição para identificar tendências de retenção e padrões de consumo. Esta etapa explora métricas essenciais, como taxa de retenção, quantidade total de pedidos, valores médios sem outliers e LTV (Lifetime Value), fornecendo uma visão detalhada da evolução e do valor dos clientes ao longo de sua jornada. A análise de coorte permite detectar comportamentos e preferências dos consumidores em diferentes períodos e ajustar estratégias para maximizar a retenção e o valor do cliente, promovendo uma gestão mais eficaz do relacionamento com os consumidores.

Para compreender melhor o comportamento de compra dos clientes e identificar variações significativas em seus padrões de consumo, é importante observar como transações que fogem do comportamento típico — ou seja, transações consideradas como outliers — se distribuem ao longo do tempo e entre diferentes coortes de clientes. Outliers, como compras com quantidades ou valores muito elevados, podem indicar comportamentos específicos, como grandes aquisições sazonais ou compras para revenda, e influenciam o perfil de consumo de determinadas coortes.

A análise dos outliers para as variáveis de 'quantity', 'unit_price' e 'total_price' permitirá identificar grupos de clientes com comportamentos de compra que se destacam em relação à maioria. Essa diferenciação ajuda a revelar padrões fora do comum que, se ignorados, podem distorcer as métricas globais e influenciar as estratégias de segmentação.

In [ ]:
# define limites de outliers para quantity, unit_price e total_price
limits = {
    'quantity': (lower_bound_quantity, upper_bound_quantity),
    'unit_price': (lower_bound_unit_price, upper_bound_unit_price),
    'total_price': (lower_bound_total_price, upper_bound_total_price)
}

# cria novas colunas para indicar "Normal" ou "Outlier" em cada categoria
for column, (lower, upper) in limits.items():
    ecommerce_dataset[f'{column}_status'] = ecommerce_dataset[column].apply(
        lambda x: 'Outlier' if x < lower or x > upper else 'Normal'
    )

In [ ]:
# agrupa o conjunto de dados por 'customer_id' e obtém a data da primeira compra de cada cliente
first_invoice_date_per_customer = regular_transactions.groupby('customer_id')['invoice_date'].min()

# renomeia a série resultante para 'first_invoice_date' para facilitar a leitura
first_invoice_date_per_customer.name = 'first_invoice_date'

# adiciona a data da primeira compra de cada cliente ao conjunto de dados original, associando pelo 'customer_id'
ecommerce_dataset = ecommerce_dataset.join(first_invoice_date_per_customer, on='customer_id')

# cria uma nova coluna 'valid_customer_month' que armazena o mês/ano da primeira compra de cada cliente, como um período mensal
ecommerce_dataset['cohort_month'] = ecommerce_dataset['first_invoice_date'].dt.to_period('M')

In [ ]:
# atualizar variável 'regular_transactions' para filtrar os registros onde 'transaction_type' é "Regular transaction" considerando os novos dados em ecommerce_dataset
regular_transactions = ecommerce_dataset[ecommerce_dataset['transaction_type'] == 'Regular transaction']

In [ ]:
# agrupa por 'cohort_month' e 'quantity_status' e conta os registros em cada grupo
cohort_quantity_status = (
    regular_transactions.groupby(['cohort_month', 'quantity_status'])
    .size()
    .reset_index(name='count')
)

# converte a coluna 'cohort_month' para string para evitar erro de serialização
cohort_quantity_status['cohort_month'] = cohort_quantity_status['cohort_month'].astype(str)

# cria o gráfico de linhas para segmentação de outliers por coorte
fig = px.line(
    cohort_quantity_status, 
    x='cohort_month', 
    y='count', 
    color='quantity_status',
    title='Segmentação de outliers por coorte - Quantity',
    labels={'cohort_month': 'Mês/Ano', 'count': 'Contagem', 'quantity_status': 'Status de Quantity'}
)

# ajusta o layout do gráfico
fig.update_layout(
    xaxis_title='Coorte (Ano/Mês)',
    yaxis_title='Contagem de Itens',
    legend_title='Status de Quantity',
    xaxis=dict(
        tickvals=cohort_quantity_status['cohort_month'],  # posição dos ticks
        ticktext=cohort_quantity_status['cohort_month'],  # texto dos ticks
        tickangle=-45  # rotaciona os ticks
    ),
    height=500,  # ajusta a altura do gráfico para visualização
    width=1100   # ajusta a largura do gráfico
)

# exibe o gráfico
fig.show()


A análise da quantidade de itens transacionados revela que os valores normais apresentam um pico acentuado em novembro de 2018, seguido de uma queda e posterior estabilização em volumes menores. Esse comportamento sugere uma demanda inicial elevada que se regulariza ao longo do tempo. Já os outliers mostram um padrão diferente, concentrando-se principalmente em novembro e dezembro de 2018, possivelmente refletindo eventos específicos ou compras excepcionais de grande volume. Embora menos frequentes, esses outliers continuam presentes ao longo do ano, sugerindo compras pontuais de alto volume que ocorrem esporadicamente.

In [ ]:
# agrupa por 'cohort_month' e 'unit_price_status' e conta os registros em cada grupo
cohort_unit_price_status = (
    regular_transactions.groupby(['cohort_month', 'unit_price_status'])
    .size()
    .reset_index(name='count')
)

# converte a coluna 'cohort_month' para string para evitar erro de serialização
cohort_unit_price_status['cohort_month'] = cohort_unit_price_status['cohort_month'].astype(str)

# cria o gráfico de linhas para segmentação de outliers por coorte - Unit Price
fig = px.line(
    cohort_unit_price_status, 
    x = 'cohort_month', 
    y = 'count', 
    color = 'unit_price_status',
    title = 'Segmentação de outliers por coorte - Unit Price',
    labels = {'cohort_month': 'Mês/Ano', 'count': 'Contagem', 'unit_price_status': 'Status de Unit Price'}
)

# ajusta o layout do gráfico
fig.update_layout(
    xaxis_title = 'Coorte (Ano/Mês)',
    yaxis_title = 'Contagem de itens',
    legend_title = 'Status de Unit Price',
    xaxis = dict(
        tickvals = cohort_unit_price_status['cohort_month'],  # posição dos ticks
        ticktext = cohort_unit_price_status['cohort_month'],  # texto dos ticks
        tickangle = -45  # rotaciona os ticks
    ),
    height = 500,  # ajusta a altura do gráfico para visualização
    width = 1100   # ajusta a largura do gráfico
)

# exibe o gráfico
fig.show()


A análise do preço unitário por coorte revela um pico expressivo de transações em novembro de 2018 para valores normais, seguido de uma queda acentuada e uma estabilização nos meses seguintes. Esse comportamento indica uma concentração inicial de vendas a preços considerados normais, com uma tendência de estabilização ao longo do tempo. No que diz respeito aos outliers, observa-se um padrão similar de redução após o pico inicial, mas em um nível significativamente menor e com um volume constante, sugerindo que transações a preços unitários fora do padrão ocorrem de forma esporádica, porém consistente. 

In [ ]:
# agrupa por 'cohort_month' e 'total_price_status' e conta os registros em cada grupo
cohort_total_price_status = (
    regular_transactions.groupby(['cohort_month', 'total_price_status'])
    .size()
    .reset_index(name='count')
)

# converte a coluna 'cohort_month' para string para evitar erro de serialização
cohort_total_price_status['cohort_month'] = cohort_total_price_status['cohort_month'].astype(str)

# cria o gráfico de linhas para segmentação de outliers por coorte - Unit Price
fig = px.line(
    cohort_total_price_status, 
    x = 'cohort_month', 
    y = 'count', 
    color = 'total_price_status',
    title = 'Segmentação de outliers por coorte - Total Price',
    labels = {'cohort_month': 'Mês/Ano', 'count': 'Contagem', 'total_price_status': 'Status de Total Price'}
)

# ajusta o layout do gráfico
fig.update_layout(
    xaxis_title = 'Coorte (Ano/Mês)',
    yaxis_title = 'Contagem de itens',
    legend_title = 'Status de Total Price',
    xaxis = dict(
        tickvals = cohort_total_price_status['cohort_month'],  # posição dos ticks
        ticktext = cohort_total_price_status['cohort_month'],  # texto dos ticks
        tickangle = -45  # rotaciona os ticks
    ),
    height = 500,  # ajusta a altura do gráfico para visualização
    width = 1100   # ajusta a largura do gráfico
)

# exibe o gráfico
fig.show()


As transações normais alcançaram um pico em novembro de 2018, seguido por uma queda significativa e uma estabilização nos meses subsequentes, refletindo um padrão mais uniforme de compra ao longo do tempo. Os outliers, por outro lado, tiveram seu maior volume em dezembro de 2018, logo após o pico das transações normais, mas rapidamente diminuíram e se mantiveram em níveis baixos e constantes. Esse padrão sugere que, enquanto a maioria das compras se estabilizou após um período inicial de alta atividade, há uma presença ocasional de transações de alto valor, que ocorrem de maneira menos frequente, mas que ainda têm impacto significativo em determinados meses.

Agora será realizada a análise dos dados com separação por coortes mensais, o que permitirá observar o comportamento dos clientes a partir do mês de sua primeira compra. Essa segmentação facilitará o entendimento das mudanças no perfil de compra ao longo do tempo e o acompanhamento de métricas específicas conforme a “idade” de cada coorte, possibilitando comparações entre diferentes grupos de clientes com base em seus meses de entrada.

In [ ]:
# criação de nova coluna para armazenar a diferença em meses entre o primeiro pedido e o pedido atual
ecommerce_dataset['cohort_age'] = ecommerce_dataset['invoice_date'].dt.to_period('M').astype('int64') - ecommerce_dataset['first_invoice_date'].dt.to_period('M').astype('int64')

In [ ]:
# atualizar variável 'regular_transactions' para filtrar os registros onde 'transaction_type' é "Regular transaction" considerando os novos dados em ecommerce_dataset
regular_transactions = ecommerce_dataset[ecommerce_dataset['transaction_type'] == 'Regular transaction']

Será feita a análise da taxa de retenção dos clientes ao longo do tempo, utilizando uma estrutura de coortes mensais. Essa abordagem permitirá acompanhar a porcentagem de clientes que permanecem ativos em meses subsequentes ao seu primeiro pedido. Assim, será possível observar a variação da retenção conforme a "idade" de cada coorte e identificar padrões de comportamento ao longo dos meses, proporcionando insights sobre a fidelização dos clientes.

In [ ]:
# agrupar e contar o número de usuários únicos por coorte e idade do coorte
cohort_data = regular_transactions.groupby(['cohort_month', 'cohort_age']).agg({'customer_id': 'nunique'}).reset_index()

# renomear a coluna
cohort_data.columns = ['cohort_month', 'cohort_age', 'n_customers']

# exibir dados 
cohort_data.head(15)

In [ ]:
# cálculo o total de usuários únicos para cada `cohort_month`
total_customer_per_month = regular_transactions.groupby('cohort_month')['customer_id'].nunique().reset_index()

# renomear a coluna para clareza
total_customer_per_month.columns = ['cohort_month', 'total_customers']

# exibir tabela
total_customer_per_month

In [ ]:
# mesclar o total de usuários com a tabela `cohort_data`
cohort_data = pd.merge(cohort_data, total_customer_per_month, on='cohort_month')

# cálculo a porcentagem de `n_customers` em relação ao total de usuários
cohort_data['percentage_users'] = (cohort_data['n_customers'] / cohort_data['total_customers']) * 100

# arredondar os valores de porcentagem para 2 casas decimais
cohort_data['percentage_users'] = cohort_data['percentage_users'].round(2)

# exibir dados
cohort_data


In [ ]:
# criação de tabela dinâmica com os dados agrupados por mês do primeiro pedido
retention_timing_analysis = cohort_data.pivot_table(
    index='cohort_month',
    columns='cohort_age',
    values='percentage_users',
    aggfunc='sum'
)

# converter o índice 'cohort_age' para string com formato desejado
retention_timing_analysis.index = retention_timing_analysis.index.strftime('%Y-%m')

# cria o heatmap usando Plotly Express
fig = px.imshow(
    retention_timing_analysis.values,
    labels=dict(x="Idade do coorte (meses)", y="Mês do primeiro pedido", color="Taxa de retenção (%)"),
    x=retention_timing_analysis.columns.astype(str),
    y=retention_timing_analysis.index,
    aspect="auto",
    text_auto=True,
    color_continuous_scale="Greens"
)

# ajusta o layout e o título
fig.update_layout(
    title="Taxa de retenção (%)",
    xaxis_title="Idade do coorte (meses)",
    yaxis_title="Mês do primeiro pedido",
    width=1100,
    height=650,
    yaxis=dict(type="category")
)

# exibe o gráfico
fig.show()

O gráfico acima apresenta a retenção de clientes ao longo do tempo, estruturada em coortes mensais. A coluna inicial para cada mês de coorte é sempre de 100%, representando a totalidade dos clientes que realizaram compras no mês inicial da coorte. Nos meses seguintes, observa-se uma queda gradual, indicando que, com o passar do tempo, menos clientes de cada coorte permanecem ativos. Esse padrão de diminuição ao longo do tempo é comum e reflete uma tendência de compras pontuais ou de curta duração, onde apenas uma fração continua a realizar pedidos regularmente.

Um ponto interessante é que, no mês de novembro de 2019, todas as coortes exibem um aumento, indicando uma possível reativação dos clientes nesse período. Esse comportamento pode estar associado a campanhas sazonais, como promoções de fim de ano ou eventos específicos que incentivam o retorno dos clientes. Além disso, algumas coortes, como as de novembro de 2018 e janeiro de 2019, apresentam percentuais ligeiramente superiores nos meses subsequentes, sugerindo um desempenho marginalmente melhor. Essa combinação de fatores pode fornecer insights importantes para otimizar futuras campanhas e entender o impacto de eventos sazonais na fidelização dos clientes.

A análise a seguir buscará calcular a quantidade de pedidos ao longo do tempo, com a separação dos dados em coortes mensais baseados no mês do primeiro pedido de cada grupo de clientes:

In [ ]:
# criação de tabela dinâmica com os dados agrupados por mês do primeiro pedido
invoice_quantity_analysis = regular_transactions.pivot_table(
    index='cohort_month',
    columns='cohort_age',
    values='invoice_no',
    aggfunc='nunique'
)

# converter o índice 'cohort_age' para string com formato desejado
invoice_quantity_analysis.index = invoice_quantity_analysis.index.strftime('%Y-%m')

# cria o heatmap usando Plotly Express
fig = px.imshow(
    invoice_quantity_analysis.values,
    labels=dict(x="Idade do coorte (meses)", y="Mês do primeiro pedido", color="Quantidade"),
    x=invoice_quantity_analysis.columns.astype(str),
    y=invoice_quantity_analysis.index,
    aspect="auto",
    text_auto=True,
    color_continuous_scale="Blues"
)

# ajusta o layout e o título
fig.update_layout(
    title="Quantidade total de pedidos",
    xaxis_title="Idade do coorte (meses)",
    yaxis_title="Mês do primeiro pedido",
    width=1100,
    height=650,
    yaxis=dict(type="category")
)

# exibe o gráfico
fig.show()

O gráfico mostra a quantidade de pedidos realizados ao longo do tempo por diferentes coortes de clientes, definidos com base no mês do primeiro pedido. Observa-se que as coortes de novembro e dezembro de 2018 têm as maiores quantidades de pedidos, especialmente nos primeiros meses subsequentes ao início da coorte. Esse comportamento é menos intenso nas coortes mais recentes, sugerindo uma possível diminuição no volume inicial de pedidos ou variações sazonais.

Um ponto notável é o aumento significativo na quantidade de pedidos em novembro de 2019, observado em praticamente todas as coortes. Esse comportamento pode indicar a influência de campanhas promocionais de fim de ano ou eventos específicos que reativaram clientes de várias coortes, impulsionando temporariamente o volume de pedidos. Além disso, como em análises de coorte tradicionais, é evidente uma tendência geral de redução na quantidade de pedidos ao longo do tempo, à medida que o engajamento dos clientes diminui após o primeiro mês de compra.

Nas próximas análises, serão desconsiderados os valores outliers do conjunto de dados e selecionados apenas os registros de transações do tipo regular. A exclusão dos outliers permite focar nas transações que representam o comportamento típico dos clientes, evitando que valores atípicos distorçam as médias e dificultem a identificação de padrões consistentes. A análise restrita às transações regulares elimina eventos pontuais e transações incomuns, proporcionando uma visão mais precisa das tendências de compra e comportamento dos clientes ao longo do tempo.

In [ ]:
# aplica filtros para remover outliers em todas as métricas:
# - filtra transações com 'quantity' dentro dos limites inferiores e superiores definidos
# - filtra transações com 'unit_price' dentro dos limites inferiores e superiores definidos
# - filtra transações com 'total_price' dentro dos limites inferiores e superiores definidos
# assim, são mantidas apenas as transações "regulares" (sem outliers) em todas as três métricas
regular_transactions_all_without_outliers = regular_transactions[
    (regular_transactions['quantity'] >= lower_bound_quantity) & (regular_transactions['quantity'] <= upper_bound_quantity) &
    (regular_transactions['unit_price'] >= lower_bound_unit_price) & (regular_transactions['unit_price'] <= upper_bound_unit_price) &
    (regular_transactions['total_price'] >= lower_bound_total_price) & (regular_transactions['total_price'] <= upper_bound_total_price)
]

A seguir será realizada a análise do valor total médio dos pedidos ao longo das diferentes coortes de clientes, tomando como referência o mês do primeiro pedido. A tabela dinâmica agrupará esses valores médios de transação por coorte mensal, permitindo visualizar como o valor médio das compras evolui entre as diferentes coortes e identificar possíveis variações sazonais ou tendências de comportamento de compra entre os clientes ao longo do tempo.

In [ ]:
# criação de tabela dinâmica com os dados agrupados por mês do primeiro pedido
mean_total_revenue_analysis = regular_transactions_all_without_outliers.pivot_table(
    index='cohort_month',
    values='total_price',
    columns='cohort_age',
    aggfunc='mean'
)

# converter o índice 'cohort_age' para string com formato desejado
mean_total_revenue_analysis.index = mean_total_revenue_analysis.index.strftime('%Y-%m')

# arredondar os valores para 2 casas decimais
mean_total_revenue_analysis = mean_total_revenue_analysis.round(2)

# cria o heatmap usando Plotly Express
fig = px.imshow(
    mean_total_revenue_analysis.values,
    labels=dict(x="Idade do coorte (meses)", y="Mês do primeiro pedido", color="Média"),
    x=mean_total_revenue_analysis.columns.astype(str),
    y=mean_total_revenue_analysis.index,
    aspect="auto",
    text_auto=True,
    color_continuous_scale="Oranges"
)

# ajusta o layout e o título
fig.update_layout(
    title="Valor total médio dos pedidos sem outliers",
    yaxis_title="Mês do primeiro pedido",
    width=1100,
    height=650,
    yaxis=dict(type="category")
)

# exibe o gráfico
fig.show()

O gráfico apresenta o valor total médio dos pedidos ao longo das coortes de clientes, excluindo os outliers, e revela um comportamento mais estável dos valores médios ao longo do tempo. No mês inicial de cada coorte, os valores variam entre 7.47 e 13.07, com um destaque para as coortes de fevereiro, maio e setembro de 2019, que apresentam os maiores valores médios iniciais. Nos meses subsequentes, observa-se uma leve variação nos valores médios, indicando uma tendência de estabilidade, sem grandes flutuações de valores entre os pedidos.

Esse comportamento pode indicar que, ao excluir os outliers, o padrão de gasto médio dos clientes é mais homogêneo ao longo do tempo, e a maioria dos pedidos tem valores próximos entre si. A estabilidade ao longo das idades das coortes sugere que, para os clientes que continuam a realizar pedidos, os valores médios tendem a permanecer consistentes. 

A análise a seguir busca identificar o comportamento do valor unitário médio dos itens adquiridos por diferentes coortes de clientes ao longo do tempo. Excluindo os outliers, será possível observar padrões e variações no valor médio unitário dos produtos em cada idade do coorte, o que pode revelar tendências de preços médios mantidos ou alterados conforme a recorrência das compras e as características de cada grupo de clientes.

In [ ]:
# criação de tabela dinâmica com os dados agrupados por mês do primeiro pedido
mean_unit_revenue_analysis = regular_transactions_all_without_outliers.pivot_table(
    index='cohort_month',
    columns='cohort_age',
    values='unit_price',
    aggfunc='mean'
)

# converter o índice 'cohort_age' para string com formato desejado
mean_unit_revenue_analysis.index = mean_unit_revenue_analysis.index.strftime('%Y-%m')

# arredondar os valores para 2 casas decimais
mean_unit_revenue_analysis = mean_unit_revenue_analysis.round(2)

# cria o heatmap usando Plotly Express
fig = px.imshow(
    mean_unit_revenue_analysis.values,
    labels=dict(x="Idade do coorte (meses)", y="Mês do primeiro pedido", color="Média"),
    x=mean_total_revenue_analysis.columns.astype(str),
    y=mean_unit_revenue_analysis.index,
    aspect="auto",
    text_auto=True,
    color_continuous_scale="Oranges"
)

# ajusta o layout e o título
fig.update_layout(
    title="Valor unitário médio dos pedidos sem outliers",
    yaxis_title="Mês do primeiro pedido",
    width=1100,
    height=650,
    xaxis=dict(visible=False),
    yaxis=dict(type="category")
)

# exibe o gráfico
fig.show()

Este gráfico ilustra a média do valor unitário dos pedidos para diferentes coortes de clientes ao longo do tempo. Observa-se que, em novembro de 2018, há valores de média unitária mais elevados, com um pico de 3.16 em seu segundo mês de coorte. Esse padrão sugere que clientes que compraram nessa época podem ter adquirido itens de valor unitário mais alto, possivelmente influenciados por campanhas sazonais ou características específicas dos produtos adquiridos.

Nos meses subsequentes, o valor unitário médio apresenta uma tendência geral de diminuição, com algumas variações entre os períodos de coorte. Ao longo de 2019, o valor médio unitário se estabiliza em torno de valores menores, em torno de 2 a 3, com exceções como agosto de 2019, que mostra um aumento considerável na média. Esse comportamento sugere que, ao longo do tempo, o perfil de produtos comprados por clientes pode ter mudado, possivelmente com foco em itens de menor valor unitário.

A análise a seguir examinará a quantidade média de itens comprados pelos clientes, distribuída por coortes mensais e sua evolução ao longo do tempo. Utilizando uma tabela dinâmica para calcular a média de itens por pedido em cada mês do primeiro pedido, busca-se compreender possíveis tendências e padrões de compra.

In [ ]:
# criação de tabela dinâmica com os dados agrupados por mês do primeiro pedido
itens_quantity_analysis = regular_transactions_all_without_outliers.pivot_table(
    index='cohort_month',
    columns='cohort_age',
    values='quantity',
    aggfunc='mean'
)

# converter o índice 'cohort_age' para string com formato desejado
itens_quantity_analysis.index = itens_quantity_analysis.index.strftime('%Y-%m')

# arredondar os valores para 2 casas decimais
itens_quantity_analysis = itens_quantity_analysis.round(2)

# cria o heatmap usando Plotly Express
fig = px.imshow(
    itens_quantity_analysis.values,
    labels=dict(x="Idade do coorte (meses)", y="Mês do primeiro pedido", color="Média"),
    x=itens_quantity_analysis.columns.astype(str),
    y=itens_quantity_analysis.index,
    aspect="auto",
    text_auto=True,
    color_continuous_scale="Oranges"
)

# ajusta o layout e o título
fig.update_layout(
    title="Quantidade média de itens",
    xaxis_title="Idade do coorte (meses)",
    yaxis_title="Mês do primeiro pedido",
    width=1100,
    height=650,
    yaxis=dict(type="category")
)

# exibe o gráfico
fig.show()

A análise do gráfico de quantidade média de itens revela uma tendência de aumento na média de itens comprados durante os meses de julho, agosto e setembro de 2019, independente da coorte de origem. Esse comportamento sugere que algum evento sazonal, promoção ou campanha estratégica pode ter influenciado positivamente as compras, resultando em um crescimento consistente no volume médio de itens adquiridos pelos clientes nesse período.

Além disso, após esse aumento, os valores começam a apresentar uma leve tendência de queda, indicando que o crescimento foi temporário e possivelmente impulsionado por um fator externo. Não há uma estabilização clara nas médias de itens entre as coortes após esse pico, reforçando a hipótese de que esse aumento foi um efeito pontual, talvez relacionado a uma ação de marketing ou outro evento específico que incentivou os clientes a comprarem mais itens em suas transações durante o terceiro trimestre de 2019.

A análise a seguir examinará o LTV (Lifetime Value) médio por coorte mensal, considerando apenas transações regulares e excluindo valores outliers. O total gasto por clientes será calculado para cada combinação de coorte e idade do coorte, seguido pela divisão desse valor pelo número de clientes únicos, resultando no LTV médio.

In [ ]:
# calcula o total gasto por 'cohort_month' e 'cohort_age' e o número de clientes únicos
ltv_data = regular_transactions_all_without_outliers.groupby(['cohort_month', 'cohort_age']).agg({
    'total_price': 'sum',  # soma o valor total gasto
    'customer_id': 'nunique'  # conta o número de clientes únicos
}).reset_index()

# calcula o ltv médio dividindo o total gasto pelo número de clientes
ltv_data['ltv'] = ltv_data['total_price'] / ltv_data['customer_id']

# pivota os dados para visualização em formato de heatmap
ltv_pivot = ltv_data.pivot(index='cohort_month', columns='cohort_age', values='ltv')

# converte o índice para string para exibir os rótulos corretamente no gráfico
ltv_pivot.index = ltv_pivot.index.strftime('%Y-%m')

# cria o heatmap com plotly express
fig = px.imshow(
    ltv_pivot.values,
    labels=dict(x="Idade do coorte (meses)", y="Mês do primeiro pedido", color="ltv"),
    x=ltv_pivot.columns.astype(str),
    y=ltv_pivot.index,
    color_continuous_scale="darkmint",
    aspect="auto",
    text_auto=".2f"  # adiciona anotações automáticas com duas casas decimais
)

# ajusta o layout e o título do gráfico
fig.update_layout(
    title="LTV médio por coorte",
    xaxis_title="Idade do coorte (meses)",
    yaxis_title="Mês do primeiro pedido",
    width=1100,
    height=600,
    yaxis=dict(type="category")  # define o eixo y como categórico para exibir todos os rótulos
)

# exibe o gráfico
fig.show()

Este gráfico apresenta o LTV médio por coorte mensal ao longo dos meses subsequentes após a primeira compra. Observa-se que a coorte de novembro de 2018 possui um LTV significativamente mais alto nos primeiros meses de idade da coorte, com um valor máximo em torno de 1464.52 no segundo e terceiro meses. Esse padrão inicial elevado é seguido por uma leve estabilização e até aumento nos meses posteriores, como em idades de 8 a 11 meses, onde o LTV atinge picos adicionais, como 1782.85 no 11º mês. Esse comportamento indica que os clientes desta coorte mantiveram uma média de valor por cliente elevada ao longo do tempo, possivelmente devido a uma base de clientes mais engajada ou a valores de transação mais altos.

Observa-se também um padrão interessante de aumento do LTV para todas as coortes entre os meses de setembro, outubro e novembro de 2019, independentemente da data de início da coorte. Esse comportamento consistente pode sugerir uma resposta positiva a fatores sazonais ou campanhas específicas ocorridas nesse período, levando a um aumento no valor médio das transações. Além disso, a maioria das coortes demonstra um crescimento gradual no LTV nos primeiros meses, seguido por uma estabilização em valores médios, com alguns picos isolados. Esse padrão indica que, após um período inicial de maior engajamento, os valores tendem a estabilizar, sugerindo um comportamento mais regular nas compras ao longo do tempo.

### Etapa 4 - Segmentação de clientes

Nesta etapa da análise, o objetivo é agrupar os produtos com base em suas descrições textuais, utilizando técnicas de processamento de linguagem natural e o método de clusterização K-means. Esse processo de segmentação visa identificar padrões e grupos de produtos com características semelhantes, a partir de palavras-chave presentes nas descrições. Para isso, será aplicada a técnica de vetorização TF-IDF para transformar as descrições em representações numéricas, e o método do cotovelo será utilizado para auxiliar na definição do número ideal de clusters.

Essa segmentação permitirá uma visão mais estruturada sobre os diferentes tipos de produtos, facilitando análises posteriores e o desenvolvimento de estratégias para personalizar ofertas e melhorar a experiência do cliente.

In [ ]:
# converte todas as descrições dos produtos para letras minúsculas na coluna 'description'
regular_transactions_all_without_outliers.loc[:, 'description'] = regular_transactions_all_without_outliers['description'].str.lower()

In [ ]:
# define suas stopwords personalizadas
custom_stop_words = ['set', 'pack', 'box', 'design', 'tins', 'piece', 'pieces', 'blue', 'red', 'pink', 'green', 'white', 'black', 'lunch', 'heart', 'vintage', 'hot', 'small', 'jumbo', 'retrospot', 'skull']

# cria uma lista de stopwords combinando as stopwords em inglês com as personalizadas
combined_stop_words = list(text.ENGLISH_STOP_WORDS.union(custom_stop_words))

# inicializa o vetor TF-IDF com as stopwords combinadas
vectorizer = TfidfVectorizer(stop_words=combined_stop_words)

# aplica a vetorização na coluna de descrições em letras minúsculas
X = vectorizer.fit_transform(regular_transactions_all_without_outliers['description'])

In [ ]:
# determinação do número de clusters com o método do cotovelo
inertia = []
range_clusters = range(2, 15)

# loop para calcular a inércia para cada número de clusters no intervalo definido
for n_clusters in range_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

# define os pontos inicial e final para traçar a linha do método do cotovelo
point1 = np.array([2, inertia[0]])  # primeiro ponto (2 clusters)
point2 = np.array([14, inertia[-1]])  # último ponto (14 clusters)

# calcula as distâncias de cada ponto de inércia até a linha traçada entre point1 e point2
distances = []
for i in range(len(inertia)):
    # coordenadas do ponto atual
    point = np.array([range_clusters[i], inertia[i]])
    
    # calcula a distância do ponto atual até a linha entre point1 e point2
    distance = np.abs(np.cross(point2 - point1, point1 - point)) / np.linalg.norm(point2 - point1)
    distances.append(distance)

# encontra o número de clusters com a maior distância, que representa o ponto ideal
optimal_clusters = range_clusters[np.argmax(distances) + 1]  # +1 devido ao índice começar em 0

# exibe o gráfico com o ponto do cotovelo marcado
plt.plot(range_clusters, inertia, marker='o', label='Inércia')
plt.xlabel('Número de clusters')
plt.ylabel('Inércia')
plt.title('Método do cotovelo com distância máxima')
plt.axvline(optimal_clusters, linestyle='--', color='red', label=f'Escolha ideal: {optimal_clusters} clusters')
plt.legend()
plt.show()

In [ ]:
# define o número de clusters baseado no gráfico do cotovelo
kmeans = KMeans(n_clusters=11, random_state=42)  # usa 11 clusters conforme identificado
kmeans.fit(X)

# garante que regular_transactions_all_without_outliers seja uma cópia independente para evitar problemas de referência
regular_transactions_all_without_outliers = regular_transactions_all_without_outliers.copy()

# adiciona os rótulos dos clusters gerados pelo k-means ao conjunto de dados
regular_transactions_all_without_outliers['product_cluster'] = kmeans.labels_

# agrupa as descrições dos produtos por cluster, concatenando todas as descrições dentro de cada cluster
clustered_descriptions = regular_transactions_all_without_outliers.groupby('product_cluster')['description'].apply(lambda x: ' '.join(x))

# define a quantidade de clusters que foi selecionada para análise
n_clusters = 11  # ajusta para o número de clusters escolhido anteriormente

In [ ]:
# define o tamanho da figura para exibir as word clouds
plt.figure(figsize=(17,12))

# loop para criar a word cloud para cada cluster
for i in range(n_clusters):
    # cria uma wordcloud para o cluster i
    wordcloud = WordCloud(width=800, height=400, background_color='white', max_words=50).generate(clustered_descriptions[i])

    # plota a word cloud
    plt.subplot(4, 3, i + 1)  # ajusta os parâmetros de subplot para se adaptar ao número de clusters
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Cluster {i}')

# exibe todas as word clouds
plt.suptitle("Principais palavras por cluster")
plt.show()

No início do processo de agrupamento, um conjunto de stopwords foi definido, excluindo termos comuns como "set", "box", "design", além de cores e adjetivos, visando remover palavras que não contribuem significativamente para o significado dos produtos.

Na sequência, o método do cotovelo foi utilizado para determinar o número ideal de clusters. O gráfico resultante indicou que 11 clusters seria o ponto ideal, onde ocorre a maior redução na taxa de inércia. Essa escolha permite uma representação mais robusta dos dados, equilibrando a complexidade dos grupos e evitando clusters excessivamente redundantes ou específicos.

Após a definição do número de clusters, o K-means foi aplicado para categorizar os produtos em 11 agrupamentos distintos, e os rótulos foram adicionados ao conjunto de dados. As descrições dos produtos em cada cluster foram consolidadas e, em seguida, visualizadas por meio de nuvens de palavras (word clouds) para facilitar a identificação dos termos mais frequentes e relevantes em cada agrupamento. A análise das word clouds resultou nas seguintes interpretações para os clusters:

- **Cluster 0 - Artigos de chá e mesa**: Inclui itens relacionados a serviços de chá e louça, com destaque para termos como "teacup", "regency" e "plate". São produtos que remetem ao estilo clássico de chá.

- **Cluster 1 - Iluminação secorativa**: Envolve produtos de decoração com foco em luzes e penduradores, com palavras como "light", "holder" e "hanging", indicando itens para iluminar e decorar ambientes.

- **Cluster 2 - Acessórios de confeitaria**: Concentra produtos para bolos e confeitaria, com termos como "cake", "cases" e "fairy", sugerindo artigos para decoração e armazenamento de doces.

- **Cluster 3 - Bolsas retrô e vintage**: Reúne itens voltados para bolsas grandes e acessórios com um estilo vintage, evidenciado por palavras como "jumbo", "bag" e "vintage".

- **Cluster 4 - Artigos para festas e celebrações**: Agrupa produtos voltados para festas, com foco em decoração e acessórios de comemoração, com palavras como "bunting", "party" e "candles".

- **Cluster 5 - Jardinagem e utensílios domésticos**: Inclui artigos para jardinagem e itens domésticos, com termos como "gardener", "kneeling", e "towel", voltados para atividades de cuidado e manutenção de jardins e lares.

- **Cluster 6 - Temática infantil e espaço**: Concentra-se em produtos infantis e temáticos de espaço, com destaque para "spaceboy", "children" e "lunch", indicando uma linha de itens voltados para crianças com tema espacial.

- **Cluster 7 - Produtos femininos e design jovem**: Abrange produtos de design voltados para o público feminino e infantil, com destaque para termos como "dolly", "girl" e "design", sugerindo itens para crianças e adolescentes.

- **Cluster 8 - Placas e artigos metálicos**: Focado em sinalização e itens decorativos de metal, com palavras como "sign", "metal" e "blue", são produtos de decoração com um estilo industrial.

- **Cluster 9 - Artigos de cozinha vintage**: Envolve itens utilitários de cozinha com um toque retrô, como "water bottle", "hot water", e "retrospot", sugerindo artigos para armazenamento e preparação de bebidas.

- **Cluster 10 - Artigos natalinos**: Focado em itens de decoração para o Natal e festas, com destaque para "christmas", "tree" e "star", agrupando produtos para celebrações de fim de ano.

Esse processo de análise permitiu a criação de categorias claras e consistentes para os produtos, possibilitando um entendimento mais profundo dos tipos de itens comercializados e das preferências dos clientes. A categorização é útil para estratégias de marketing direcionadas, assim como para a organização do catálogo de produtos, facilitando a personalização de ofertas com base nos interesses específicos dos consumidores.

O gráfico abaixo apresenta a distribuição de clientes entre os diferentes clusters de produtos. O gráfico exibe a quantidade de clientes em cada categoria, o que permite identificar os grupos de produtos mais populares e orientar estratégias de marketing e estoque.

In [ ]:
# dicionário com as descrições simples de cada cluster
cluster_descriptions = {
    '0': 'Cluster 0 - Artigos de chá e mesa',
    '1': 'Cluster 1 - Iluminação decorativa',
    '2': 'Cluster 2 - Acessórios de confeitaria',
    '3': 'Cluster 3 - Bolsas retrô e vintage',
    '4': 'Cluster 4 - Artigos para festas e celebrações',
    '5': 'Cluster 5 - Jardinagem e utensílios domésticos',
    '6': 'Cluster 6 - Temática infantil e espaço',
    '7': 'Cluster 7 - Produtos femininos e design jovem',
    '8': 'Cluster 8 - Placas e artigos metálicos',
    '9': 'Cluster 9 - Artigos de cozinha vintage',
    '10': 'Cluster 10 - Artigos natalinos'
}

In [ ]:
# conta a quantidade de clientes em cada cluster de produto e cria um DataFrame com essa contagem
product_cluster_counts = regular_transactions_all_without_outliers['product_cluster'].value_counts().reset_index()
product_cluster_counts.columns = ['product_clusters', 'count']

# substitui os rótulos numéricos dos clusters pelos nomes descritivos das categorias de produto
product_cluster_counts['product_clusters_label'] = product_cluster_counts['product_clusters'].astype(str).map(cluster_descriptions)

# converte a coluna de clusters numéricos para string para facilitar a categorização no gráfico
product_cluster_counts['product_clusters'] = product_cluster_counts['product_clusters'].astype(str)

# cria o gráfico de barras com as categorias de produtos, incluindo uma legenda interativa
fig = px.bar(product_cluster_counts, 
             x='product_clusters', 
             y='count', 
             title='Distribuição das categorias de clientes',
             labels={'count': 'Quantidade de clientes', 'product_clusters_label': 'Categoria de produto'},
             color='product_clusters_label', 
             text='count')

# ajusta o layout do gráfico para melhorar a visualização e facilitar a interpretação
fig.update_layout(
    xaxis_title='Categoria de produto',
    yaxis_title='Quantidade de clientes',
    legend_title_text='Categoria de produto',
    height=600,  # Define a altura do gráfico
    width=1300   # Define a largura do gráfico
)

# exibe o gráfico
fig.show()

O gráfico mostra uma concentração expressiva de clientes no Cluster 9, com mais de 290.000 clientes, destacando-se como a categoria mais representativa. Esse domínio sugere que os produtos desse cluster possuem alta demanda ou apelo entre os consumidores, possivelmente indicando preferência ou recorrência.

Os demais clusters apresentam uma distribuição muito menor, com o Cluster 3 em segundo lugar (cerca de 36.000 clientes) e o Cluster 10 logo em seguida, com aproximadamente 20.000. Os outros clusters variam entre 5.000 e 18.000 clientes, sugerindo um interesse mais disperso ou nichado nesses produtos. Essa variação oferece uma visão clara sobre quais clusters podem ser priorizados em estratégias de marketing e estoque.

A próxima análise irá identificar o cluster de produto predominante para cada cliente, baseando-se na frequência de compra:

In [ ]:
# calcula a frequência de compra de cada cliente por cluster de produto
cliente_produto_cluster = regular_transactions_all_without_outliers.groupby(['customer_id', 'product_cluster']).size().reset_index(name='purchase_count')

# identifica o cluster de produto predominante para cada cliente, encontrando o cluster mais comprado por cada cliente
cliente_cluster_dominante = cliente_produto_cluster.loc[cliente_produto_cluster.groupby('customer_id')['purchase_count'].idxmax()]

# renomeia a coluna para representar o cluster predominante
cliente_cluster_dominante = cliente_cluster_dominante.rename(columns={'product_cluster': 'dominant_product_cluster'})

# atribui o cluster de produto ao cliente, juntando essa informação ao DataFrame dos clientes para uma visualização completa
clientes_com_cluster = regular_transactions_all_without_outliers[['customer_id']].drop_duplicates().merge(
    cliente_cluster_dominante[['customer_id', 'dominant_product_cluster']],
    on='customer_id',
    how='left'
)

In [ ]:
# conta a quantidade de clientes em cada cluster de produto dominante e cria um DataFrame com essa contagem
dominant_cluster_counts = clientes_com_cluster['dominant_product_cluster'].value_counts().reset_index()
dominant_cluster_counts.columns = ['product_clusters', 'count']

# substitui os rótulos numéricos dos clusters pelos nomes descritivos das categorias de produto
dominant_cluster_counts['product_clusters_label'] = dominant_cluster_counts['product_clusters'].astype(str).map(cluster_descriptions)

# converte a coluna de clusters numéricos para string para facilitar a categorização no gráfico
dominant_cluster_counts['product_clusters'] = dominant_cluster_counts['product_clusters'].astype(str)

# cria o gráfico de barras com as categorias de produtos, incluindo uma legenda interativa
fig = px.bar(dominant_cluster_counts, 
             x='product_clusters', 
             y='count', 
             title='Distribuição dos clusters de produtos dominantes entre os clientes',
             labels={'count': 'Quantidade de clientes', 'product_clusters_label': 'Categoria de produto'},
             color='product_clusters_label', 
             text='count')

# ajusta o layout do gráfico para melhorar a visualização e facilitar a interpretação
fig.update_layout(
    xaxis_title='Categoria de produto',
    yaxis_title='Quantidade de clientes',
    legend_title_text='Categoria de produto',
    height=600,  # Define a altura do gráfico
    width=1300   # Define a largura do gráfico
)

# exibe o gráfico
fig.show()


O gráfico acima mostra a distribuição dos clusters de produtos dominantes entre os clientes, revelando uma clara predominância do Cluster 9, que é associado a 3.810 clientes. Esse número é expressivamente maior em comparação com os demais clusters, que possuem números significativamente menores. O segundo cluster mais representado é o Cluster 3, com apenas 129 clientes, enquanto todos os outros clusters apresentam menos de 50 clientes cada.

Essa análise converge com o observado no gráfico anterior, onde o Cluster 9 também se destacava em quantidade. Esse padrão indica que a maioria dos clientes possui um comportamento de compra fortemente concentrado em um tipo específico de produto, sugerindo uma preferência clara por produtos deste cluster.

A próxima etapa da análise visa segmentar os clientes com base em características comportamentais utilizando o modelo RFM (Recency, Frequency, Monetary). A intenção é calcular as variáveis RFM para cada cliente e, em seguida, aplicar o método de clusterização K-means para agrupar clientes com perfis de compra semelhantes. O número ideal de clusters será determinado pelo método do cotovelo, permitindo uma divisão otimizada dos clientes em grupos distintos. Após a segmentação, serão calculadas as médias de recência, frequência e valor monetário para cada cluster, bem como o número de clientes por grupo.

In [ ]:
# define a data de referência como a data mais recente no conjunto de dados
referencia_data = regular_transactions_all_without_outliers['invoice_date'].max()

# calcula a recência, frequência e valor monetário para cada cliente
rfm = regular_transactions_all_without_outliers.groupby('customer_id').agg({
    'invoice_date': lambda x: (referencia_data - x.max()).days,  # calcula a recência em dias a partir da última compra
    'invoice_no': 'count',  # conta o número de transações para obter a frequência de compras
    'total_price': 'sum'  # soma o total das compras para calcular o valor monetário
}).rename(columns={'invoice_date': 'recency', 'invoice_no': 'frequency', 'total_price': 'monetary'}).reset_index()

In [ ]:
# inicializa o StandardScaler para normalizar os dados de RFM
scaler = StandardScaler()

# aplica o scaler para padronizar as métricas de recência, frequência e valor monetário e transforma os valores para uma escala com média 0 e desvio padrão 1
rfm_scaled = scaler.fit_transform(rfm[['recency', 'frequency', 'monetary']])

In [ ]:
# inicializa a lista para armazenar as inércias
inertia = []
range_clusters = range(2, 10)  # testa de 2 a 10 clusters

# calcula a inércia para cada número de clusters no intervalo definido
for n_clusters in range_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(rfm_scaled)
    inertia.append(kmeans.inertia_)

# define os pontos inicial e final para traçar a linha do método do cotovelo
point1 = np.array([2, inertia[0]])  # primeiro ponto (2 clusters)
point2 = np.array([9, inertia[-1]])  # último ponto (9 clusters)

# calcula as distâncias de cada ponto de inércia até a linha traçada entre point1 e point2
distances = []
for i in range(len(inertia)):
    point = np.array([range_clusters[i], inertia[i]])
    distance = np.abs(np.cross(point2 - point1, point1 - point)) / np.linalg.norm(point2 - point1)
    distances.append(distance)

# identifica o número ideal de clusters (ponto mais distante da linha)
optimal_clusters = range_clusters[np.argmax(distances) + 1]  # +1 para ajustar o índice

# exibe o gráfico com o ponto do cotovelo marcado
plt.plot(range_clusters, inertia, marker='o', label='inércia')
plt.xlabel('Número de clusters')
plt.ylabel('Inércia')
plt.title('Método do cotovelo com distância máxima')
plt.axvline(optimal_clusters, linestyle='--', color='red', label=f'Escolha ideal: {optimal_clusters} clusters')
plt.legend()
plt.show()

In [ ]:
# inicializa o modelo KMeans com o número de clusters determinado pelo método do cotovelo
kmeans = KMeans(n_clusters=4, random_state=42)  # ajuste o número de clusters conforme o resultado do cotovelo

# aplica o modelo KMeans nos dados escalados e atribui o rótulo de cluster a cada cliente no DataFrame rfm
rfm['customer_cluster'] = kmeans.fit_predict(rfm_scaled)

# exibe a tabela
rfm.head()

In [ ]:
# agrupa os dados RFM por customer_cluster e calcula a média de recency, frequency, e monetary para cada cluster
# além disso, conta a quantidade de clientes em cada cluster
rfm_summary = rfm.groupby('customer_cluster').agg(
    recency=('recency', 'mean'),          # calcula a média de recency para cada cluster
    frequency=('frequency', 'mean'),       # calcula a média de frequency para cada cluster
    monetary=('monetary', 'mean'),         # calcula a média de monetary para cada cluster
    customer_count=('customer_id', 'count')      # conta o número de clientes em cada cluster
).reset_index()  # reinicia o índice para que 'cluster' seja uma coluna

# exibe a tabela
rfm_summary

A análise dos clusters fornece uma visão segmentada do comportamento dos clientes, com base em recência, frequência e valor monetário das compras. O Cluster 0 destaca-se como o grupo mais leal e ativo, caracterizado pela menor média de recência e valores elevados tanto em frequência quanto em monetário. Esses clientes realizam compras frequentes e têm um ticket médio alto, indicando que são extremamente valiosos para a empresa e devem ser alvo de estratégias de fidelização. Em contrapartida, o Cluster 1 inclui clientes com alta recência e valores baixos em frequência e monetário, sugerindo um perfil de clientes menos engajados, que fazem poucas compras e com baixo valor agregado. Esse grupo pode beneficiar-se de ações de reativação, incentivando maior engajamento e frequência de compras.

O Cluster 2 é um caso atípico, contendo um único cliente que apresenta valores extremos em todas as métricas, principalmente no valor monetário, sugerindo uma compra isolada de alto valor. Esse cliente, por seu comportamento excepcional, foi separado em um cluster único, o que destaca a importância de monitorar transações pontuais de alto valor, já que podem distorcer a análise dos outros grupos. Além disso, é possível observar que o Cluster 2 agrupou os clientes que foram categorizados anteriormente com 'costumer_id' 0, ou seja, aqueles que estavam com valor de ID ausente possivelmente relacionado a compras rápidas. Por fim, o Cluster 3 inclui clientes com métricas intermediárias, refletindo um comportamento moderado em relação à frequência e valor de compras. Esses clientes apresentam engajamento regular, mas ainda com potencial para aumento de frequência e valor médio gasto.

A sequência de códigos a seguir visa categorizar os clientes com base em suas características de Recência, Frequência e Valor Monetário (RFM), e depois visualizar essa categorização.

In [ ]:
# função para categorizar os clusters com base nos valores médios específicos de cada cluster
def categorize_cluster(row):
    cluster_avg = rfm_summary.loc[row['customer_cluster']]
    
    # define as regras de categorização considerando as médias específicas de cada cluster
    if row['recency'] <= cluster_avg['recency'] and row['frequency'] >= cluster_avg['frequency'] and row['monetary'] >= cluster_avg['monetary']:
        return 'Clientes leais premium'
    elif row['recency'] <= cluster_avg['recency'] and row['frequency'] >= cluster_avg['frequency'] and row['monetary'] < cluster_avg['monetary']:
        return 'Clientes leais econômicos'
    elif row['recency'] <= cluster_avg['recency'] and row['frequency'] < cluster_avg['frequency'] and row['monetary'] >= cluster_avg['monetary']:
        return 'Clientes novos de alto valor'
    elif row['recency'] <= cluster_avg['recency'] and row['frequency'] < cluster_avg['frequency'] and row['monetary'] < cluster_avg['monetary']:
        return 'Clientes novos econômicos'
    elif row['recency'] > cluster_avg['recency'] and row['frequency'] >= cluster_avg['frequency'] and row['monetary'] >= cluster_avg['monetary']:
        return 'Clientes em risco de alto valor'
    elif row['recency'] > cluster_avg['recency'] and row['frequency'] >= cluster_avg['frequency'] and row['monetary'] < cluster_avg['monetary']:
        return 'Clientes em risco econômicos'
    elif row['recency'] > cluster_avg['recency'] and row['frequency'] < cluster_avg['frequency'] and row['monetary'] >= cluster_avg['monetary']:
        return 'Clientes pontuais de alto valor'
    elif row['recency'] > cluster_avg['recency'] and row['frequency'] < cluster_avg['frequency'] and row['monetary'] < cluster_avg['monetary']:
        return 'Clientes pontuais de econômicos'
    elif row['recency'] > cluster_avg['recency'] and row['frequency'] < cluster_avg['frequency'] and row['monetary'] >= cluster_avg['monetary']:
        return 'Clientes com potencial para engajamento'
    else:
        return 'Outros'

# aplica a função de categorização ao DataFrame rfm para criar uma nova coluna "categoria" para cada cliente
rfm['categoria'] = rfm.apply(categorize_cluster, axis=1)

O código apresentado define uma função para categorizar clientes com base nos valores médios de Recência (quantos dias desde a última compra), Frequência (número de compras) e Valor Monetário (total gasto) específicos de cada cluster. Abaixo, explicam-se as categorias criadas e os critérios utilizados para sua classificação:

- **Clientes leais premium**: representam clientes que compram com frequência e têm um valor monetário elevado, além de uma baixa recência, ou seja, realizaram compras recentemente. Este grupo é valioso para a empresa, pois demonstra um alto nível de lealdade e alto poder de compra.

- **Clientes leais econômicos**: são clientes que também compram com frequência, mas gastam menos por transação, embora continuem comprando regularmente (baixa recência). Esses clientes mantêm um relacionamento constante, mas com um ticket médio mais baixo.

- **Clientes novos de alto valor**: clientes que iniciaram a relação recentemente (baixa recência), têm uma frequência de compras menor, mas realizam transações de alto valor. Este perfil pode representar clientes que estão testando a empresa com um alto ticket inicial.

- **Clientes novos econômicos**: clientes que também iniciaram a relação recentemente, mas com uma frequência de compra e um valor monetário mais baixos. São clientes em fase inicial de engajamento, ainda com potencial de crescimento, mas atualmente menos rentáveis.

- **Clientes em risco de alto valor**: clientes que compraram com alta frequência e com valor monetário elevado, mas que têm uma alta recência (há mais tempo sem comprar). Esse grupo é importante de ser monitorado, pois representa clientes de alto valor que podem estar migrando para a inatividade.

- **Clientes em risco econômicos**: clientes com frequência razoável de compras e de ticket médio mais baixo, mas com alta recência. Indica clientes de menor valor que estão em risco de se tornarem inativos.

- **Clientes pontuais de alto valor**: realizam compras esporádicas, mas, quando compram, possuem um valor monetário elevado. Este grupo apresenta alto valor em transações individuais, mas não se relaciona de forma consistente com a empresa.

- **Clientes pontuais econômicos**: clientes com baixa frequência de compra e ticket médio mais baixo, além de alta recência. Esses clientes fazem compras esporádicas e têm menor impacto no faturamento.

- **Clientes com potencial para engajamento**: embora tenham uma recência alta (estão afastados), esses clientes possuem valores de frequência e monetário superiores à média, sugerindo que podem ser reengajados com as campanhas certas. Representam uma oportunidade de resgatar clientes com potencial de retorno.

- **Outros**: categoria de fallback para clientes que não se enquadram claramente nos critérios anteriores. Essa categoria pode servir como um ponto de revisão para ajustar as regras ou investigar características adicionais que diferenciem esses clientes.

Essas categorias permitem uma segmentação detalhada dos clientes, auxiliando na criação de estratégias de retenção, reengajamento e fidelização específicas para cada perfil.

Abaixo será gerada a visualização da distribuição de clientes por categoria, contando o número de clientes em cada grupo e gerando um gráfico de barras horizontal:

In [ ]:
# conta a quantidade de clientes em cada categoria
categoria_counts = rfm['categoria'].value_counts().reset_index()
categoria_counts.columns = ['categoria', 'quantidade']

# cria o gráfico de barras horizontal
fig = px.bar(categoria_counts, 
             y='categoria', 
             x='quantidade', 
             title='Distribuição das categorias de clientes',
             labels={'quantidade': 'Quantidade de clientes', 'categoria': 'Categoria'},
             color='categoria', 
             orientation='h',
             text='quantidade')

# ajusta o layout para melhorar a visualização
fig.update_layout(
    yaxis_title='Categoria',
    xaxis_title='Quantidade de clientes',
    height=500,  # ajusta a altura do gráfico para visualização
    width=1100   # ajusta a largura do gráfico
)

fig.show()

O gráfico de distribuição das categorias de clientes revela a predominância de certos perfis entre os consumidores analisados. A categoria "Clientes pontuais econômicos" possui o maior número, com 1.298 clientes, seguida de "Clientes novos econômicos" com 1.261. Essas categorias indicam clientes que compram com pouca frequência e/ou com menor gasto monetário, o que sugere a presença de consumidores que realizam compras pontuais ou estão iniciando um relacionamento com a empresa, mas com um ticket médio menor.

Por outro lado, "Clientes leais premium", com 694 clientes, representa um grupo significativo de consumidores recorrentes e com alto valor de compra, evidenciando um núcleo de clientes fiéis e valiosos para o negócio. As outras categorias, como "Clientes em risco de alto valor" e "Clientes leais econômicos", possuem menos representatividade, o que sugere nichos mais específicos com diferentes comportamentos de compra, possivelmente demandando estratégias de retenção e engajamento para manter esses clientes ativos e incentivar um aumento de valor em categorias de risco econômico e engajamento.

### Etapa 5 - Testes de hipóteses estatísticas

Nesta etapa, serão realizados testes de hipóteses estatísticas para investigar padrões e relações comportamentais nos dados de compra. O objetivo é verificar, com base em evidências estatísticas, a validade de hipóteses formuladas sobre comportamentos específicos dos clientes, como a relação entre o tamanho do pedido e a probabilidade de devolução, a influência de devoluções na retenção de clientes, a sazonalidade nas vendas e a correlação entre o valor total gasto e a quantidade de itens comprados por pedido. 

A 1ª hipótese a ser testada tem como objetivo entender melhor o comportamento de compra e devolução dos clientes. O teste foca na relação entre o tamanho dos pedidos e a probabilidade de devoluções, conforme detalhado abaixo:

**Hipótese 1**:
- **H0 (Hipótese nula)**: Clientes que fazem pedidos maiores (em quantidade de itens) não têm uma probabilidade maior de realizar devoluções. Ou seja, o tamanho do pedido não influencia a chance de devolução.

- **H1 (Hipótese alternativa)**: Clientes que fazem pedidos maiores (em quantidade de itens) têm uma probabilidade maior de realizar devoluções. Ou seja, pedidos maiores estão associados a uma maior chance de devolução.

In [ ]:
# filtra transações de devoluções no conjunto de dados original
returns_transactions = ecommerce_dataset[ecommerce_dataset['transaction_type'] == 'Credit']

# calcula a quantidade média de itens por pedido para cada cliente em transações regulares
mean_items_regular = regular_transactions.groupby('customer_id')['quantity'].mean().reset_index()
mean_items_regular.columns = ['customer_id', 'avg_items_regular']

# calcula a quantidade média de itens por pedido para cada cliente em devoluções (valores negativos de quantidade)
mean_items_returns = returns_transactions.groupby('customer_id')['quantity'].mean().abs().reset_index()
mean_items_returns.columns = ['customer_id', 'avg_items_returns']

# combina os dois DataFrames para comparar as médias entre transações regulares e devoluções
mean_items_merged = mean_items_regular.merge(mean_items_returns, on='customer_id', how='inner')

In [ ]:
# teste de Shapiro-Wilk para normalidade
# verifica a normalidade para a média de itens em transações regulares
shapiro_stat_reg, p_value_reg = st.shapiro(mean_items_merged['avg_items_regular'])

# verifica a normalidade para a média de itens em devoluções
shapiro_stat_ret, p_value_ret = st.shapiro(mean_items_merged['avg_items_returns'])

# exibe os resultados do teste de Shapiro-Wilk para transações regulares
print("Teste de Shapiro-Wilk para transações regulares:")
print("Estatística:", shapiro_stat_reg)
print("Valor p:", p_value_reg)

# exibe os resultados do teste de Shapiro-Wilk para devoluções
print("\nTeste de Shapiro-Wilk para devoluções:")
print("Estatística:", shapiro_stat_ret)
print("Valor p:", p_value_ret)

# interpretação do teste de Shapiro-Wilk
if p_value_reg < 0.05:
    print("\nOs dados de 'avg_items_regular' não seguem uma distribuição normal.")
else:
    print("\nOs dados de 'avg_items_regular' seguem uma distribuição normal.")

if p_value_ret < 0.05:
    print("Os dados de 'avg_items_returns' não seguem uma distribuição normal.")
else:
    print("Os dados de 'avg_items_returns' seguem uma distribuição normal.")

In [ ]:
# teste de Mann-Whitney para comparar as médias de itens entre pedidos regulares e pedidos com devolução
# como os dados não seguem uma distribuição normal, o teste de Mann-Whitney é apropriado

t_stat, p_value = mannwhitneyu(mean_items_merged['avg_items_regular'], 
                               mean_items_merged['avg_items_returns'], 
                               alternative='greater')

# exibe as médias de itens para pedidos regulares e pedidos com devolução
print("Média de itens para pedidos regulares:", mean_items_merged['avg_items_regular'].mean())
print("Média de itens para pedidos com devolução:", mean_items_merged['avg_items_returns'].mean())

# exibe o valor p do teste de Mann-Whitney
print("Valor p:", p_value)

# interpretação do resultado
if p_value < 0.05:
    print("Rejeitamos a hipótese nula: pedidos maiores têm uma maior probabilidade de devolução.")
else:
    print("Não rejeitamos a hipótese nula: não há evidência suficiente para afirmar que pedidos maiores têm maior probabilidade de devolução.")

O teste de Shapiro-Wilk foi realizado para verificar a normalidade dos dados de média de itens comprados por pedido em transações regulares e devoluções. Os resultados indicaram que ambas as distribuições não seguem uma normalidade, conforme evidenciado pelos valores-p extremamente baixos, o que levou à rejeição da hipótese de normalidade para ambas as variáveis.

Dado que os dados não seguem uma distribuição normal, foi aplicado o teste de Mann-Whitney para comparar as médias de itens entre pedidos regulares e pedidos com devolução. A hipótese de pesquisa propôs que pedidos com uma quantidade maior de itens teriam uma probabilidade maior de resultar em devoluções. O resultado do teste de Mann-Whitney apresentou um valor-p bem abaixo de 0,05, permitindo rejeitar a hipótese nula. Isso sugere que, estatisticamente, pedidos maiores estão associados a uma probabilidade mais alta de serem devolvidos, suportando a hipótese de que clientes com pedidos maiores em quantidade de itens têm maior probabilidade de realizar devoluções.

A 2ª hipótese a ser testada busca analisar o impacto das devoluções na retenção de clientes, explorando se há uma correlação entre o comportamento de devolução e a probabilidade de o cliente deixar de fazer compras. O objetivo é entender se clientes que realizaram devoluções estão mais propensos a sair do que aqueles que nunca devolveram itens, conforme detalhado abaixo:

**Hipótese 2**:

- **H0 (Hipótese nula)**: Clientes que realizam devoluções não têm uma probabilidade significativamente maior de sair do que clientes que nunca realizaram devoluções. Em outras palavras, a devolução de produtos não influencia a probabilidade de saída dos clientes.

- **H1 (Hipótese alternativa)**: Clientes que realizam devoluções têm uma probabilidade significativamente maior de sair do que clientes que nunca realizaram devoluções. Isso indica que o comportamento de devolução estaria associado a uma tendência maior de abandono por parte dos clientes.

In [ ]:
# define a coluna 'churn_risk' com base na categoria RFM
rfm['churn_risk'] = rfm['categoria'].apply(lambda x: 1 if 'em risco' in x or 'pontuais' in x else 0)

# exibe as primeiras linhas para verificar
rfm.head()

In [ ]:
# filtra transações de devoluções
returns = ecommerce_dataset[ecommerce_dataset['transaction_type'] == 'Credit']

# cria uma coluna indicadora para devoluções no DataFrame `rfm`
rfm['has_return'] = rfm['customer_id'].isin(returns['customer_id']).astype(int)

# calcula a proporção de clientes com devoluções em risco de churn e sem risco
churn_return_counts = rfm.groupby(['churn_risk', 'has_return']).size().unstack(fill_value=0)
churn_return_counts.columns = ['No Return', 'Has Return']

# exibe a tabela de contagem
churn_return_counts

In [ ]:
# executa o teste qui-quadrado
chi2, p, _, _ = chi2_contingency(churn_return_counts)

# exibe o resultado
print("Teste Qui-Quadrado:")
print("Estatística:", chi2)
print("Valor p:", p)

# interpretação do resultado
if p < 0.05:
    print("Rejeitamos a hipótese nula: clientes em risco de churn têm uma maior probabilidade de realizar devoluções.")
else:
    print("Não rejeitamos a hipótese nula: não há evidência suficiente para afirmar que clientes em risco de churn têm maior probabilidade de devoluções.")

A classificação de churn foi realizada utilizando a segmentação de clientes baseada nas métricas RFM (Recência, Frequência e Valor Monetário). No modelo RFM, os clientes foram categorizados em diferentes grupos de acordo com seu comportamento de compra, e categorias específicas foram associadas a um risco de churn maior.

Neste caso, clientes classificados como "em risco" foram identificados com base em sua categoria no modelo RFM. Categorias que indicavam baixo engajamento recente, como "Clientes em risco econômicos" e "Clientes pontuais de baixo valor", foram atribuídas ao grupo de clientes em risco de churn. Esse grupo representa aqueles clientes que apresentam maior chance de deixar de comprar, com base no padrão de comportamento observado.

Para operacionalizar essa classificação no código, foi criada uma nova coluna churn_risk no DataFrame `rfm`, onde os clientes foram atribuídos com o valor 1 se pertenciam a categorias de alto risco de churn, e 0 caso contrário. Essa classificação foi essencial para o teste de hipótese, pois permitiu investigar se havia uma associação entre o risco de churn e a ocorrência de devoluções, usando o teste estatístico de Qui-Quadrado.

O teste mostrou que há uma associação significativa entre o status de churn e a prática de devoluções, indicando que os clientes com maior risco de churn tendem a realizar devoluções com mais frequência, o que pode ser um indicador relevante para estratégias de retenção de clientes.

A 3ª hipótese a ser testada visa explorar a sazonalidade nas vendas, verificando se existem períodos específicos do ano com um aumento significativo nas vendas devido a promoções ou outros fatores sazonais. O teste buscará identificar padrões de aumento nas vendas, especialmente durante meses conhecidos por suas promoções, como novembro e dezembro, e como esses períodos impactam o comportamento de compra dos clientes. A análise será baseada nos dados de vendas agregados por período e as diferenças significativas entre os meses serão avaliadas para confirmar ou refutar a presença de sazonalidade.

**Hipótese 3**:

- **H0 (Hipótese nula)**: As vendas não apresentam variações significativas em determinados períodos do ano. Ou seja, não há impacto sazonal nas vendas.
- **H1 (Hipótese alternativa)**: As vendas aumentam em certos períodos do ano, como novembro e dezembro, devido a promoções ou outros fatores sazonais.

In [ ]:
# agrega as vendas totais por mês usando a coluna 'invoice_date_month'
monthly_sales = regular_transactions.groupby('invoice_date_month')['total_price'].sum().reset_index()
monthly_sales.columns = ['month', 'total_sales']

# converte a coluna 'month' para string
monthly_sales['month'] = monthly_sales['month'].astype(str)

# separa as vendas de novembro e dezembro para comparação
holiday_sales = monthly_sales[monthly_sales['month'].str.contains(r'-10|-11|-12', na=False)]['total_sales']

# separa as vendas de todos os outros meses que não são novembro e dezembro
non_holiday_sales = monthly_sales[~monthly_sales['month'].str.contains(r'-10|-11|-12', na=False)]['total_sales']

In [ ]:
# teste de Shapiro-Wilk para normalidade
# verifica a normalidade para as vendas nos meses de novembro e dezembro
shapiro_stat_holiday, p_value_holiday = st.shapiro(holiday_sales)

# verifica a normalidade para as vendas nos outros meses
shapiro_stat_non_holiday, p_value_non_holiday = st.shapiro(non_holiday_sales)


# exibe os resultados do teste de Shapiro-Wilk para novembro e dezembro
print("Teste de Shapiro-Wilk para novembro e dezembro:")
print("Estatística:", shapiro_stat_holiday)
print("Valor p:", p_value_holiday)

# exibe os resultados do teste de Shapiro-Wilk para os outros meses
print("\nTeste de Shapiro-Wilk para os outros meses:")
print("Estatística:", shapiro_stat_non_holiday)
print("Valor p:", p_value_non_holiday)

# interpretação do teste de Shapiro-Wilk
if p_value_holiday < 0.05:
    print("\nOs dados de 'holiday_sales' (novembro e dezembro) não seguem uma distribuição normal.")
else:
    print("\nOs dados de 'holiday_sales' (novembro e dezembro) seguem uma distribuição normal.")

if p_value_non_holiday < 0.05:
    print("Os dados de 'non_holiday_sales' (outros meses) não seguem uma distribuição normal.")
else:
    print("Os dados de 'non_holiday_sales' (outros meses) seguem uma distribuição normal.")

In [ ]:
# teste t de Student para comparar as vendas nos meses de novembro e dezembro com os outros meses
# como os dados seguem uma distribuição normal, o teste t de Student é apropriado

stat, p_value = st.ttest_ind(holiday_sales, non_holiday_sales, alternative='greater')

# exibe as médias de vendas para os meses de novembro e dezembro e para os outros meses
print("Média de vendas em novembro e dezembro:", holiday_sales.mean())
print("Média de vendas nos outros meses:", non_holiday_sales.mean())

# exibe o valor p do teste t de Student
print("Valor p:", p_value)

# interpretação do resultado
if p_value < 0.05:
    print("Rejeitamos a hipótese nula: as vendas aumentam significativamente em novembro e dezembro.")
else:
    print("Não rejeitamos a hipótese nula: não há evidência suficiente para afirmar que as vendas aumentam significativamente em novembro e dezembro.")

A análise estatística da Hipótese 3 foi realizada por meio do teste de normalidade de Shapiro-Wilk e do teste t de Student. O objetivo foi verificar se há uma diferença significativa entre as vendas médias de novembro e dezembro em comparação com os outros meses do ano.

Inicialmente, o teste de Shapiro-Wilk foi aplicado para avaliar a normalidade das distribuições de vendas nos períodos de novembro/dezembro e nos demais meses. Com base nos valores de p (> 0,05 para ambos os grupos), constatou-se que os dados seguem uma distribuição normal, possibilitando o uso do teste t de Student para a comparação entre as médias.

O teste t de Student, então, avaliou a hipótese de que as vendas aumentam significativamente em novembro e dezembro em comparação com os outros meses. O valor de p obtido (0,318) indica que não há evidência estatística suficiente para rejeitar a hipótese nula. Dessa forma, conclui-se que, embora o gráfico de Arrecadação mensal apresentado anteriormente indique uma elevação aparente nas vendas de novembro, essa variação não é estatisticamente significativa em relação aos outros meses, sugerindo que não há um impacto sazonal relevante nas vendas ao longo do ano.

A análise de sazonalidade das vendas seria fortalecida com a inclusão de um maior número de registros mensais completos, especialmente considerando que os dados de novembro de 2018 e dezembro de 2019 são parciais e não refletem o volume total do mês. Essa limitação reduz a precisão das conclusões, pois dados incompletos podem não capturar totalmente o comportamento de consumo característico desses períodos. A obtenção de dados mais abrangentes e contínuos ao longo de múltiplos anos permitiria identificar padrões sazonais de forma mais robusta, evitando que flutuações pontuais ou atípicas interfiram na interpretação dos picos de vendas em meses como novembro e dezembro.

A 4ª hipótese será focada em investigar a relação entre o valor total gasto por pedido e a quantidade de itens comprados. Este teste visa identificar se há uma correlação significativa entre o tamanho do pedido, em termos de itens, e o gasto total associado, o que pode revelar tendências de comportamento de compra em pedidos de maior valor ou volume.

**Hipótese 4**:

- **H0 (Hipótese nula)**: Não existe uma correlação significativa entre o valor total gasto e a quantidade de itens comprados por pedido. Ou seja, o número de itens não influencia o valor total gasto no pedido.

- **H1 (Hipótese alternativa)**: Existe uma correlação significativa entre o valor total gasto e a quantidade de itens comprados por pedido. Ou seja, o número de itens influencia o valor total gasto no pedido.

In [ ]:
# filtra as colunas necessárias para a análise
data_for_corr = regular_transactions[['total_price', 'quantity']].dropna()

In [ ]:
# verifica a normalidade de 'total_price' usando o teste de Kolmogorov-Smirnov
kstest_stat_price, kstest_p_value_price = kstest(data_for_corr['total_price'], 'norm', 
                                                 args=(data_for_corr['total_price'].mean(), data_for_corr['total_price'].std()))
print("Teste de Kolmogorov-Smirnov para total_price:")
print("Estatística:", kstest_stat_price)
print("Valor p:", kstest_p_value_price)

# verifica a normalidade de 'quantity' usando o teste de Kolmogorov-Smirnov
kstest_stat_quantity, kstest_p_value_quantity = kstest(data_for_corr['quantity'], 'norm', 
                                                       args=(data_for_corr['quantity'].mean(), data_for_corr['quantity'].std()))
print("\nTeste de Kolmogorov-Smirnov para quantity:")
print("Estatística:", kstest_stat_quantity)
print("Valor p:", kstest_p_value_quantity)

# interpretação dos testes de Kolmogorov-Smirnov
if kstest_p_value_price < 0.05:
    print("\nOs dados de 'total_price' não seguem uma distribuição normal.")
else:
    print("\nOs dados de 'total_price' seguem uma distribuição normal.")

if kstest_p_value_quantity < 0.05:
    print("Os dados de 'quantity' não seguem uma distribuição normal.")
else:
    print("Os dados de 'quantity' seguem uma distribuição normal.")

In [ ]:
# calcula a correlação de Spearman entre total_price e quantity (devido à não-normalidade)
correlation, p_value = spearmanr(data_for_corr['total_price'], data_for_corr['quantity'])

# exibe os resultados da correlação de Spearman
print("\nCoeficiente de correlação de Spearman:", correlation)
print("Valor p:", p_value)

# interpretação do resultado
if p_value < 0.05:
    print("Rejeitamos a hipótese nula: existe uma correlação significativa entre o valor total gasto e a quantidade de itens comprados.")
else:
    print("Não rejeitamos a hipótese nula: não há evidência suficiente para afirmar que existe uma correlação significativa entre o valor total gasto e a quantidade de itens comprados.")

Para testar a Hipótese 4 foram realizadas análises de normalidade e, em seguida, um teste de correlação adequado aos dados.

Inicialmente, foi aplicado o teste de Kolmogorov-Smirnov (K-S) para verificar a normalidade das variáveis total_price e quantity. Esse teste foi escolhido como alternativa ao Shapiro-Wilk devido ao grande volume de dados, tornando-o mais apropriado e eficiente para amostras grandes. Os resultados do teste indicaram que ambas as variáveis não seguem uma distribuição normal (p-valores próximos de 0), o que justificou o uso de um teste de correlação que não depende da normalidade dos dados.

Diante da ausência de normalidade, optou-se pelo teste de correlação de Spearman, adequado para avaliar associações monotônicas sem a exigência de linearidade ou normalidade nos dados. O teste revelou um coeficiente de correlação de aproximadamente 0,67, o que representa uma correlação positiva moderada a forte entre total_price e quantity. Esse coeficiente sugere que, conforme a quantidade de itens comprados aumenta, o valor total gasto tende a crescer de forma consistente.

Além disso, o valor p foi igual a 0, indicando que essa correlação é estatisticamente significativa. Em termos práticos, rejeita-se a hipótese nula (H0) de que não há correlação significativa entre o valor total e a quantidade de itens, aceitando-se a hipótese alternativa (H1), que confirma a influência da quantidade de itens no valor total gasto por pedido.

Esse resultado é consistente com expectativas intuitivas, dado que é razoável esperar que pedidos com maior número de itens correspondam a um gasto total mais elevado.

### Etapa 6 - Conclusões e recomendações

Esta etapa final tem o objetivo de consolidar as descobertas obtidas ao longo da análise e de fornecer diretrizes práticas para a aplicação desses insights. Após investigar os padrões de comportamento de compra, as tendências de vendas sazonais, a segmentação de clientes e outros aspectos relevantes, esta seção sintetiza os principais resultados e oferece sugestões para ações estratégicas. As recomendações são formuladas com base nas evidências coletadas e visam apoiar a tomada de decisão orientada a dados, aprimorando o relacionamento com o cliente, otimizando os processos internos e potencializando o crescimento sustentável do negócio.

**Conclusões**:

- **Tendência de crescimento, sazonalidade e impacto de eventos**: A análise exploratória dos dados revelou uma tendência geral de crescimento nas vendas da Everything Plus ao longo de 2019, com picos em novembro, provavelmente impulsionados por eventos sazonais e promocionais como a Black Friday. A queda em dezembro pode ser atribuída à coleta parcial de dados nesse mês.

- **Valores ausentes relevantes**: A análise inicial dos dados revelou uma quantidade significativa de valores ausentes, principalmente nas colunas 'CustomerID' e 'Description'. Aproximadamente 25% dos registros não possuíam identificação do cliente, o que sugere a ocorrência de transações sem a devida coleta dessa informação. Já os valores ausentes em 'Description' representaram uma parcela menor (0.27%), sendo tratados com o preenchimento "unknown" para manter a integridade do conjunto de dados.

- **Transações com valores negativos**: A presença de valores negativos nas colunas 'Quantity' e 'UnitPrice' demandou uma investigação detalhada. As análises revelaram que as faturas com número iniciado por "C" indicavam transações de crédito, devoluções ou descontos, impactando negativamente o estoque e as vendas. Foram criadas categorias na coluna 'transaction_type' para classificar esses diferentes tipos de transações, incluindo "Credit", "Damaged/Returned Stock" e "Debt Adjustment".

- **Distribuição de quantidades de itens**: A análise da distribuição de quantidades de itens por pedido demonstrou que a maioria dos pedidos se concentra em volumes menores, com a mediana de 3 itens. A presença de outliers, no entanto, mostrou que algumas transações atingiram volumes extremos, chegando a quase 81.000 itens, indicando grande variação nos pedidos.

- **Padrões de preços unitários**: A análise dos preços unitários revelou que a maioria dos produtos possui valores acessíveis, concentrados entre 1 e 4 unidades monetárias, com mediana em 2. No entanto, a presença de outliers com valores unitários extremos, atingindo cerca de 40.000 unidades monetárias, sugere a necessidade de investigar a natureza desses valores, que podem representar erros, ajustes financeiros ou itens de alto valor.

- **Variação no valor total dos pedidos**: A distribuição do valor total dos pedidos, desconsiderando outliers, mostrou que a maioria das transações se concentra em valores entre 3 e 16 unidades monetárias, com mediana em 8.32. A inclusão dos outliers, porém, revelou uma dispersão considerável, com alguns pedidos atingindo valores extremamente altos, como 160 mil unidades monetárias, destacando a presença de compras de alto valor.

- **Produtos mais vendidos x receita gerada**: A análise dos produtos mais vendidos por quantidade e por receita revelou padrões interessantes. Enquanto produtos de baixo valor unitário, como WORLD WAR 2 GLIDERS ASSTD DESIGNS, dominam a lista de mais vendidos em quantidade, itens com maior valor unitário ou custos adicionais, como DOTCOM POSTAGE e REGENCY CAKESTAND 3 TIER, lideram a receita, mostrando a importância de considerar ambos os aspectos.

- **Comportamento de compra ao longo do tempo**: A análise de coorte, considerando o mês da primeira compra, forneceu insights sobre a evolução do comportamento de compra. Observou-se que as coortes de novembro e dezembro de 2018 apresentaram as maiores quantidades de pedidos, especialmente nos primeiros meses. Além disso, um aumento significativo na quantidade de pedidos foi observado em novembro de 2019 em quase todas as coortes, sugerindo a influência de campanhas promocionais de fim de ano.

- **Tendência de aumento na média de itens**: A análise da quantidade média de itens por coorte revelou um aumento consistente em julho, agosto e setembro de 2019, independentemente da coorte. Essa tendência sugere a influência de eventos sazonais ou campanhas promocionais nesse período, impulsionando a compra de mais itens por transação.

- **LTV e padrões de fidelização**: A análise do LTV médio por coorte destacou a coorte de novembro de 2018 com um LTV significativamente mais alto nos primeiros meses, indicando um grupo de clientes com maior valor ao longo do tempo. Observou-se também um aumento generalizado do LTV em setembro, outubro e novembro de 2019, reforçando a influência de fatores sazonais ou campanhas nesse período.

- **Segmentação de clientes**: A aplicação do modelo RFM permitiu a segmentação de clientes em grupos distintos com base em seus comportamentos de compra, revelando perfis como "Clientes leais premium", "Clientes em risco de alto valor" e "Clientes pontuais econômicos". Essa segmentação possibilita a personalização de estratégias de marketing e comunicação para cada grupo.

- **Clusterização de produtos**: A clusterização de produtos com base nas descrições textuais, utilizando o método K-means, resultou na identificação de 11 categorias de produtos distintos, como "Artigos de chá", "Iluminação e decoração" e "Itens natalinos". Essa categorização facilita a organização do catálogo de produtos, a análise de tendências de compra e a personalização de ofertas.

- **Concentração de clientes em um cluster de produtos específico**: A análise da distribuição de clientes entre os clusters de produtos revelou uma concentração expressiva no Cluster 9, associado a itens natalinos. Essa predominância sugere alta demanda ou apelo por esses produtos, demandando atenção especial em relação a estoque e estratégias de marketing.

- **Pedidos maiores e probabilidade de devolução**: O teste de hipóteses confirmou que pedidos maiores, em termos de quantidade de itens, estão estatisticamente associados a uma probabilidade maior de devoluções. Essa descoberta sugere a necessidade de investigar as causas por trás das devoluções em pedidos volumosos e implementar medidas para minimizar esse problema, como políticas de devolução mais claras ou embalagens aprimoradas.

- **Impacto das devoluções no risco de churn**: O teste Qui-Quadrado evidenciou uma associação significativa entre clientes em risco de churn e a realização de devoluções. Essa descoberta reforça a importância de monitorar o comportamento de devolução como um potencial indicador de insatisfação e risco de perda de clientes.

- **Correlação entre quantidade de itens e valor total do pedido**: O teste de correlação de Spearman confirmou a existência de uma correlação positiva moderada a forte entre o valor total gasto e a quantidade de itens comprados por pedido. Essa relação, estatisticamente significativa, indica que a quantidade de itens no pedido é um fator relevante para prever o valor total da transação.

**Recomendações para retenção de clientes**:

- **Campanhas de fidelização para clientes leais premium**: Oferecer programas de fidelidade com recompensas exclusivas, descontos personalizados e acesso antecipado a promoções para o grupo de "Clientes leais premium".

- **Estratégias de reativação para clientes em risco**: Desenvolver campanhas de email marketing direcionadas, com ofertas especiais e incentivos para a recompra, visando reativar "Clientes em risco de alto valor" e "Clientes em risco econômicos". A investigação das causas das devoluções e a implementação de medidas para solucionar os problemas identificados podem contribuir para a redução do risco de churn.

- **Engajamento de clientes pontuais**: Criar campanhas de marketing de conteúdo, com dicas e informações relevantes sobre os produtos, para aumentar o interesse e a frequência de compra dos "Clientes pontuais de alto valor" e "Clientes pontuais econômicos".

- **Ofertas personalizadas por cluster de produto**: Desenvolver campanhas de marketing direcionadas, com base nos clusters de produtos identificados, para oferecer produtos e promoções relevantes aos interesses de cada grupo de clientes. Por exemplo, para clientes do cluster "Artigos de chá", oferecer promoções em conjuntos de chá e acessórios relacionados.

- **Abordagem diferenciada para clientes com customer ID ausente**: Implementar medidas para incentivar o cadastro completo dos clientes durante a compra, visando reduzir a quantidade de transações sem 'customer_id'. Para os clientes já categorizados como Cluster 2, realizar análises complementares e considerar estratégias específicas para esse grupo, levando em conta suas características únicas.

- **Monitoramento constante das métricas RFM**: Acompanhar as métricas RFM dos clientes periodicamente para identificar mudanças nos seus comportamentos de compra e ajustar as estratégias de retenção e engajamento de acordo com a evolução dos perfis.

- **Melhoria da experiência de compra para pedidos grandes**: Oferecer opções de frete grátis ou descontos progressivos para pedidos acima de um determinado valor ou quantidade de itens, visando incentivar compras maiores e minimizar a insatisfação relacionada a devoluções em pedidos volumosos.
